### Вместо точек -- запятые!

In [1]:
#Locale settings
import locale
# Set to German locale to get comma decimal separater
locale.setlocale(locale.LC_NUMERIC, "deu_DEU")

import numpy as np
import matplotlib.pyplot as plt
plt.rcdefaults()

# Tell matplotlib to use the locale we set above
plt.rcParams['axes.formatter.use_locale'] = True

### Запуск QtLab

In [4]:
%matplotlib qt5
from qsweepy import *
from qsweepy.instruments import *
from qsweepy.save_pkl import save_pkl
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import interactive
interactive(True)

In [5]:
#Параметры кубитов.all keys should be str
qubits = {'1': {'r':{
                    'Fr': 6.599e9, #6.59783e9,
                    'Fr_min': 6.598e9,#6.575e9,
                    'Fr_max': 6.603e9,#6.62e9,
                    'dFr': 0.02e6,#0.1e6, # tolerance of resonator frequency
                     'P': -65 },
                'q':{
                    "F":
                     {'F01_min':4.985e9},#5.505e9}, # was 5.508e9,last 5.51812
                     'P':
                     {'nndac_coil_id':6,#0,
                      'awg_tek_coil_channel':1,
                      'coil_device_channel':1,
                     'P':None,    
                     #'Fmin_coil_currents':{0:1.74, 2:-0.300, 4:0.040}#"""2.02"""
                    #'Fmin_coil_currents':{0:1.826, 2:-0.000, 4:0.000}#"""2.02"""
                    #'Fmin_coil_currents':{0:1.820, 2:-0.050, 4:0.000}#"""2.02"""
                      #'Fmin_coil_currents':{0:1.814, 2:-0.100, 4:0.000}#"""2.02"""
                     #'Fmin_coil_currents':{0:1.79, 2:-0.200, 4:0.000}#"""2.02"""
                    #'Fmin_coil_currents':{0:1.246, 2:-0.00, 4:-2.90}#"""2.02"""
                      #'Fmin_coil_currents':{0:1.33, 2:-1.5, 4:-2.50},
                      #'Fmin_coil_currents':{0:-1.6, 2:0, 4:-0},
                      #'Fmin_coil_currents':{0:-1.71, 2:0.0, 4:0.0} ,
                      'Fmin_coil_currents':{0:-1.65, 2:0.0, 4:0.0}, #1809-001-2-revolution
                      'Fmin_coil_currents':{6:-1.51, 8:-0.0, 10:0.0}, #1808-072-1-revolution
                      'Fmin_coil_currents':{6:-1.6, 8:-0.4, 10:0.0}, #1808-072-1-revolution
                      'Fmin_coil_currents':{6:1.261, 8:-1.486, 10:-1.974}, #1808-072-1-revolution
                      'Fmin_coil_currents':{6:1.500, 8:-0.223, 10:-2.16}, #1808-072-1-revolution
                      'Fmin_coil_currents':{6:1.52699178, 8:-0.08319558, 10:-2.29904575}
                      #'Fmin_coil_currents':{6:1.574, 8:-0.262, 10:-2.389}, #1808-072-1-revolution   
                      #'Fmin_coil_currents':{6:-1.69, 8:-0.58703473, 10:-0.77449362},
                      #'Fmin_coil_currents':{6:-1.51, 8:0.2017048, 10:-0.66385044},
                      #'Fmin_coil_currents':{6:1.334, 8:-1.19877866, 10:-1.82},
                     }
                    }},
          '2': {'r':{
                    'Fr': 6.879e9, #6.8791e9,
                    'Fr_min': 6.878e9,#6.734e9,#6.879e9,#6.82e9,
                    'Fr_max': 6.883e9,#6.788e9,#6.884e9,#6.92e9,
                    'dFr': 0.02e6,#0.1e6, # tolerance of resonator frequency
                     'P': -65 },
                'q':{
                    "F":
                    {'F01_min':5.010e9},#{'F01_min':5.519e9},#5.52025
                    'P':
                     {
                     'nndac_coil_id':8,#2,
                     'awg_tek_coil_channel':2,
                     'coil_device_channel':2,
                     'P':None, 
                     #'Fmin_coil_currents':{0:0.0, 2: -1.80, 4:-0.0}
                    #'Fmin_coil_currents':{0:1.39, 2: 1.61, 4:0.0}
                     #'Fmin_coil_currents':{0:0, 2:-1.737, 4:0},
                     'Fmin_coil_currents':{6:0, 8:-1.6, 10:0}, #1808-072-1-revolution
                     'Fmin_coil_currents':{6:2.83358946, 8:-1.2104548 , 10:-0.67273619},
                     'Fmin_coil_currents':{6:-0.1087136, 8:-1.200, 10:0.81756234},
                     'Fmin_coil_currents':{6:-0.61722937, 8:-2.67, 10:-2.4849594},
                     'Fmin_coil_currents':{6:1.261, 8:-1.486, 10:-1.974}, #1808-072-1-revolution
                     'Fmin_coil_currents':{6:1.527, 8:-0.083, 10:-2.500}, #1808-072-1-revolution
                     'Fmin_coil_currents':{6:1.574, 8:-0.262, 10:-2.389}, #1808-072-1-revolution    
                     'Fmin_coil_currents':{6:1.500, 8:-0.223, 10:-2.16}, #1808-072-1-revolution
                     'Fmin_coil_currents':{6:1.52699178, 8:-0.08319558, 10:-2.29904575}
                     }
                }},
          '3': {'r':{
                    'Fr': 6.734e9,#'Fr': 6.7335e9,
                    'Fr_min': 6.733e9,#6.72e9,
                    'Fr_max': 6.737e9,#6.76e9,
                     'dFr': 0.02e6,#0.1e6, # tolerance of resonator frequency
                     'P': -65 },
                'q':{
                    "F":
                    {'F01_min':4.833e9},#{'F01_min':5.45525e9},#5.45678
                    'P':
                     {
                    'nndac_coil_id':10,#4,
                    'awg_tek_coil_channel':3,
                    'coil_device_channel':3,
                     'P':None, 
                    #'Fmin_coil_currents':{0:0.0, 2:0.0, 4:-2.585},
                    'Fmin_coil_currents':{6:0.0, 8:0.0, 10:-3.3},
                    'Fmin_coil_currents':{ 6:2.98395593, 8:-1.22890925, 10:-1.1},
                    #'Fmin_coil_currents':{6:1.334, 8:-1.19877866, 10:-1.82},
                    #'Fmin_coil_currents':{0:1.35, 2:-1.5, 4:-2.458}   
                    'Fmin_coil_currents':{6:-0.61722937, 8:-2.67, 10:-2.4849594},
                    'Fmin_coil_currents':{6:1.261, 8:-1.486, 10:-1.974}, #1808-072-1-revolution
                    'Fmin_coil_currents':{6:1.527, 8:-0.083, 10:-2.500}, #1808-072-1-revolution
                    'Fmin_coil_currents':{6:1.574, 8:-0.262, 10:-2.389}, #1808-072-1-revolution     
                    'Fmin_coil_currents':{6:1.500, 8:-0.223, 10:-2.16}, #1808-072-1-revolution
                    'Fmin_coil_currents':{6:1.52699178, 8:-0.08319558, 10:-2.29904575}
                     }
                }}
         }

L = np.asarray([[0.292, -0.055, 0.016], [-0.060, 0.268, 0.035], [-0.019, 0.0, 0.178]])
I0 = np.asarray([0.118, 0.274, 0.215])
Lpi1_inv = np.asarray([])

L = np.asarray([[ 2.432, -0.476,  -0.144], [ 0.583 ,  2.178, -1.084], [-0.689,  0.473,  0.989  ]])
I0 = np.asarray([-0.002, -0.004, 0.002])
 

qubit_id = '3'
switch_to_sample = {'1':4, '2':3}
sample_name = '1808-072-2-2QGA1'
coil_control_channels = [1,2,3]
#it should be dumped too
#qubits_dump = {"qubits":qubit_params,'switch':switch_to_sample}
#qjson.dump('qubits','qubit_params',qubits_dump)

In [6]:
def set_qubit_fluxes(fluxes):
    zero_flux = np.asarray([L[i,i]*I0[i] for i in range(len(qubits))])
    fluxes_corrected = fluxes+zero_flux
    currents = np.linalg.solve(L, fluxes_corrected)
    
    for qubit_id_numerical, qubit_name in enumerate(['1', '2', '3']):
        qubit_coil_id = qubits[qubit_name]['q']['nndac_coil_id']
        dac.set_voltage(currents[qubit_id_numerical], channel=qubit_coil_id)
        
    print(currents, fluxes_corrected)

In [7]:
#current_src1 = Yokogawa_GS210(address='GPIB::1::INSTR')


# Спектроскопия

In [8]:
# RF switch for making sure we know what sample we are measuring
pna = Agilent_N5242A('pna', address = 'TCPIP0::10.20.61.48::inst0::INSTR')
lo1 = Agilent_E8257D('lo1', address = 'TCPIP0::10.20.61.59::inst0::INSTR')
rf_switch = nn_rf_switch('rf_switch', address='10.20.61.224')
CHASSIS = 0
SLOT_AWG1 = 3
SLOT_AWG2 = 5
adc = instruments.Spectrum_M4i22xx('adc')

awg1 = Keysight_M3202A_S('awg1', CHASSIS, SLOT_AWG1)
awg2 = Keysight_M3202A_S('awg2', CHASSIS, SLOT_AWG2)

awg_tek = Tektronix_AWG5014('awg_tek', address = 'TCPIP0::10.20.61.186::inst0::INSTR')

sa = Agilent_N9030A('pxa', address = 'TCPIP0::10.20.61.56::inst0::INSTR')
lo_ex = lo1
lo_ro = pna
coil_device = awg_tek

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)


In [7]:
awg1.set_offset(0, channel=3)
awg1.set_amplitude(0, channel=3)
awg1.set_waveform([0]*50000, channel=3)
awg1.set_output(1, channel=3)
awg1.run()

In [8]:
dac = nndac('TCPIP0::10.20.61.12::1000::SOCKET')

VisaIOError: VI_ERROR_RSRC_NFOUND (-1073807343): Insufficient location information or the requested device or resource is not present in the system.

# Mixer bypass

In [ ]:
dac.set_voltage(qubits[qubit_id]['q']['P']['Fmin_coil_currents'][qubits[qubit_id]['q']['P']['nndac_coil_id']],
                channel=qubits[qubit_id]['q']['P']['nndac_coil_id'])

In [8]:
# когда спектроскопия не рабтает и всё плохо, тыкай сюды!
# вектнй анлизатр цепей - в режим свипа по частоте
pna.set_sweep_mode("LIN")
# смесители - открыть
#awg_tek = awg
#for channel in range(0,4):
#awg._visainstrument.write('AWGC:DOUT1 OFF')
#awg._visainstrument.write('AWGC:DOUT2 OFF')
#awg._visainstrument.write('AWGC:DOUT3 OFF')
#awg._visainstrument.write('AWGC:DOUT4 OFF')
awg2.trigger_source_types = [0,6,6,6]
awg2.trigger_source_channels = [0, 4000,4000,4000]
awg2.trigger_delays = [40, 0,0,0]
awg2.trigger_behaviours = [0,4,4,4]
awg1.trigger_source_types = [6,6,6,6]
awg1.trigger_source_channels = [4000, 4000,4000,4000]
awg1.trigger_delays = [0, 0,0,0]
awg1.trigger_behaviours = [4,4,4,4]
awg2.set_trigger_mode(0)
for channel in range(0,4):
    awg2.set_amplitude(0.2, channel=channel)
    awg2.set_offset(0.2,channel=channel)
    awg2.set_output(1, channel=channel)
    awg2.set_waveform(waveform = [0]*20000, channel=channel)
for channel in range(0,4):
    awg1.set_amplitude(0.2, channel=channel)
    awg1.set_offset(0.2,channel=channel)
    awg1.set_output(1, channel=channel)
    awg1.set_waveform(waveform = [0]*20000, channel=channel)
for channel in range(1,5):
    awg_tek.set_amplitude(0.2, channel=channel)
    awg_tek.set_offset(0.0,channel=channel)
    awg_tek.set_output(1, channel=channel)
    awg_tek.set_waveform([0]*20000, channel=channel)
awg2.set_marker(length=5, delay=0, channel=0, pxi_channels=1)
awg1.set_marker(length=5, delay=0, channel=0, pxi_channels=2)
awg2.run()
awg1.run()
#     

# Задаём постояннmtые смещения на каналы 1 и 3
#awg_tek.set_waveform([0]*awg_tek.get_nop(), channel=1)
#awg_tek.set_waveform([0]*awg_tek.get_nop(), channel=2)
#awg_tek.set_waveform([0]*awg_tek.get_nop(), channel=3)
#awg_tek.set_waveform([0]*awg_tek.get_nop(), channel=4)

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)


# Резонаторная спектроскопия

### Одномерный скан при фиксированном токе 

## Overview

In [9]:
lo1.set_status(0)
pna.set_status(1)
pna.set_power(-45) # 
pna.set_nop(1001)
pna.set_bandwidth(2000)
pna.set_xlim(6.5e9, 7.0e9)
#pna.set_bandwidth(100)
freqs = pna.get_freqpoints()
S21 = pna.get_tracedata()

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)


## All anticrossings in same scan

In [10]:
#pna.get_points()['S-parameter'][0][1]
pna.set_power(5)
pna.set_bandwidth(200)
pna.pre_sweep()
plt.plot(pna.get_points()['S-parameter'][0][1],np.log10(np.abs(pna.measure()['S-parameter']))*10)
pna.post_sweep()

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)


In [14]:
if coil_device == awg_tek:
    raise ValueError('awg_tek is the active coil control device. This cell is for nndac.')
#coil = 'z1'
#current_max = 1.5e-3
voltage_max = 1
current_nop = 11
nndac_channels = [6,8,10]#np.arange(0, 24)#[21,1,7]
#currents=np.linspace(-current_max,current_max,current_nop)
voltages=np.linspace(-voltage_max,voltage_max,current_nop)
#current_src1.set_status(1)
pna.set_nop(1001)
pna.set_power(-20)
pna.set_bandwidth(2000)
pna.set_power(qubits[qubit_id]['r']['P']+65) 
for nndac_channel_id in nndac_channels:
    #set_qubit_fluxes([0,0,0])
    def set_abstract_voltage(x):
        
        dac.set_voltage(x, channel=nndac_channel_id)
        dac.set_voltage(-x, channel=2)
    dac.set_voltage(0.05, channel=2)
    coil = 'NNDAC-'+str(nndac_channel_id)
    pna.set_xlim(6.5e9,7.0e9)
    result = sweep.sweep(pna, 
                    (voltages, #lambda x: dac.set_voltage(x, channel=nndac_channel_id), 
                     set_abstract_voltage, 'Coil #{} voltage'.format(coil)), 
                    filename='Sample 1808-072-2 revolution Coil #{} anticrossings everything in max'.format(coil))
    dac.set_voltage(0, channel=nndac_channel_id)

ValueError: awg_tek is the active coil control device. This cell is for nndac.

In [ ]:
plt.close('all')

## Resonator control (N*M) anticrossing scan

In [15]:
if coil_device == awg_tek:
    raise ValueError('awg_tek is the active coil control device. This cell is for nndac.')
#coil = 'z1'
#current_max = 1.5e-3
voltage_max = 4
current_nop = 101
nndac_channels = [6,8,10]#[18,20]#[18, 20, 16]#[21,1,7]
awg_tek_channels = [1,2,3]
#currents=np.linspace(-current_max,current_max,current_nop)
voltages=np.linspace(-voltage_max,voltage_max,current_nop)
#current_src1.set_status(1)
pna.set_nop(401)
pna.set_bandwidth(500)
pna.set_power(qubits[qubit_id]['r']['P']+55) 
lo1.set_status(0)
for nndac_channel_id in nndac_channels:
    for qubit_id in ['1', '2', '3']:
        coil = 'NNDAC-'+str(nndac_channel_id)
        
        pna.set_nop((qubits[qubit_id]['r']['Fr_max']-qubits[qubit_id]['r']['Fr_min'])/qubits[qubit_id]['r']['dFr']+1)
        pna.set_xlim(qubits[qubit_id]['r']['Fr_min'], qubits[qubit_id]['r']['Fr_max'])
        result = sweep.sweep(pna, 
                        (voltages, lambda x: dac.set_voltage(x, channel=nndac_channel_id), 'Coil #{} voltage, qubit{}'.format(coil,qubit_id)), 
                        filename='1808_072-2 Coil #{} anticrossings qubit {}'.format(coil, qubit_id))
        dac.set_voltage(0, channel=6)#0)
        dac.set_voltage(0, channel=8)#2)
        dac.set_voltage(0, channel=10)#4)
lo1.set_status(1)

ValueError: awg_tek is the active coil control device. This cell is for nndac.

In [28]:
if coil_device != awg_tek:
    raise ValueError('awg_tek is not the active coil control device. This cell is for awg_tek.')
#coil = 'z1'
#current_max = 1.5e-3
voltage_max = 0.6
current_nop = 101
nndac_channels = [6,8,10]#[18,20]#[18, 20, 16]#[21,1,7]
awg_tek_channels = [1,2,3]
coil_device_channels = [1,2,3]
#currents=np.linspace(-current_max,current_max,current_nop)
voltages=np.linspace(-voltage_max,voltage_max,current_nop)
#current_src1.set_status(1)
pna.set_nop(401)
pna.set_bandwidth(500)
pna.set_power(qubits[qubit_id]['r']['P']+10) 
lo1.set_status(0)
for coil_device_channel in coil_device_channels:
    for qubit_id in ['1', '2', '3']:
        coil = 'coil_device_channel-'+str(coil_device_channel)
        coil_device.set_output(1, channel=coil_device_channel)
        pna.set_nop((qubits[qubit_id]['r']['Fr_max']-qubits[qubit_id]['r']['Fr_min'])/qubits[qubit_id]['r']['dFr']+1)
        pna.set_xlim(qubits[qubit_id]['r']['Fr_min'], qubits[qubit_id]['r']['Fr_max'])
        result = sweep.sweep(pna, 
                        (voltages, lambda x: coil_device.set_offset(x, channel=coil_device_channel), 'Coil #{} voltage, qubit{}'.format(coil,qubit_id)), 
                        filename=sample_name+' Coil #{} anticrossings qubit {}'.format(coil, qubit_id))
        #dac.set_voltage(0, channel=6)#0)
        #dac.set_voltage(0, channel=8)#2)
        #dac.set_voltage(0, channel=10)#4)
        coil_device.set_output(0, channel=coil_device_channel)
        for coil_device_channel2 in coil_device_channels:
            coil_device.set_offset(0, channel=coil_device_channel2)
lo1.set_status(1)

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)


HTML(value='')

Started at:  Oct 23 2018 09:52:40


C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)


Exiting sweep main thread #{} via return 8300

Elapsed time: 0 h 0 m 55.6 s


...

HTML(value='')

Started at:  Oct 23 2018 10:00:09


C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)


Exiting sweep main thread #{} via return 9596

Elapsed time: 0 h 0 m 42.7 s


True

## anticrossings with two simultaneous applied coils for non-diagonal coils

In [168]:
imp.reload(sweep)

<module 'qsweepy.sweep' from 'C:\\qtlab_replacement\\qsweepy\\sweep.py'>

In [10]:
if coil_device != awg_tek:
    raise ValueError('awg_tek is not the active coil control device. This cell is for awg_tek.')

anticrossing_matrix = []
files_matrix_location = []
voltage_max = 0.7
current_nop = 401
coil_device_channels = [1,2,3]
voltages=np.linspace(-voltage_max,voltage_max,current_nop)
pna.set_nop(401)
pna.set_bandwidth(500)
pna.set_power(qubits[qubit_id]['r']['P']+10) 
lo1.set_status(0)
for coil_device_channel in coil_device_channels:
    anticrossing_row = []
    files_location_row = []
    for qubit_id in ['1', '2', '3']:
        # if we are in the main coil for the current qubit, sweep only the main coil.
        if qubits[qubit_id]['q']['P']['coil_device_channel'] == coil_device_channel:
            coil = 'coil_device_channel-('+str(coil_device_channel)+')'
            def setter(x):
                coil_device.set_offset(x, channel=coil_device_channel)
        # if we are probing a non-diagonal matrix element (non-main coil), sweep the main simultaneously.
        # this helps to identify the sign of the mutual matrix element.
        else:
            coil = 'coil_device_channel-('+str(coil_device_channel)+')+('+\
                                           str(qubits[qubit_id]['q']['P']['coil_device_channel'])+')'
            coil_device.set_output(1, channel=qubits[qubit_id]['q']['P']['coil_device_channel'])
            def setter(x):
                coil_device.set_offset(x, channel=coil_device_channel)
                coil_device.set_offset(x, channel=qubits[qubit_id]['q']['P']['coil_device_channel'])
        
        coil_device.set_output(1, channel=coil_device_channel)
        pna.set_nop((qubits[qubit_id]['r']['Fr_max']-qubits[qubit_id]['r']['Fr_min'])/qubits[qubit_id]['r']['dFr']+1)
        pna.set_xlim(qubits[qubit_id]['r']['Fr_min'], qubits[qubit_id]['r']['Fr_max'])
        result,loc = sweep.sweep(pna, 
                        (voltages, setter, '#{} voltage, qubit{}'.format(coil,qubit_id)), 
                        filename=sample_name+' #{} anticrossings qubit {}'.format(coil, qubit_id),loc=True)
#         loc+=sample_name+' #{} anticrossings qubit {}'.format(coil, qubit_id)+'.pkl'
        files_location_row.append(loc)
        anticrossing_row.append(result['S-parameter'])
        
        
        for coil_device_channel2 in coil_device_channels:
            coil_device.set_offset(0, channel=coil_device_channel2)
    anticrossing_matrix.append(anticrossing_row)
    files_matrix_location.append(files_location_row)
lo1.set_status(1)

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)


HTML(value='')

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)


Started at:  Oct 24 2018 09:42:41
Exiting sweep main thread #{} via return 4516

Elapsed time: 0 h 0 m 55.8 s


HTML(value='')

...

HTML(value='')

Started at:  Oct 24 2018 09:50:11


C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)


Exiting sweep main thread #{} via return 4144

Elapsed time: 0 h 0 m 42.95 s


True

In [11]:
files_matrix_location

[['c:\\data/2018-10-24/09-42-41-1808-072-2-2QGA1 #coil_device_channel-(1) anticrossings qubit 1/1808-072-2-2QGA1 #coil_device_channel-(1) anticrossings qubit 1.pkl',
  'c:\\data/2018-10-24/09-43-41-1808-072-2-2QGA1 #coil_device_channel-(1)+(2) anticrossings qubit 2/1808-072-2-2QGA1 #coil_device_channel-(1)+(2) anticrossings qubit 2.pkl',
  'c:\\data/2018-10-24/09-44-40-1808-072-2-2QGA1 #coil_device_channel-(1)+(3) anticrossings qubit 3/1808-072-2-2QGA1 #coil_device_channel-(1)+(3) anticrossings qubit 3.pkl'],
 ['c:\\data/2018-10-24/09-45-27-1808-072-2-2QGA1 #coil_device_channel-(2)+(1) anticrossings qubit 1/1808-072-2-2QGA1 #coil_device_channel-(2)+(1) anticrossings qubit 1.pkl',
  'c:\\data/2018-10-24/09-46-26-1808-072-2-2QGA1 #coil_device_channel-(2) anticrossings qubit 2/1808-072-2-2QGA1 #coil_device_channel-(2) anticrossings qubit 2.pkl',
  'c:\\data/2018-10-24/09-47-26-1808-072-2-2QGA1 #coil_device_channel-(2)+(3) anticrossings qubit 3/1808-072-2-2QGA1 #coil_device_channel-(2)+(3)

In [50]:
import locale
# Set to German locale to get comma decimal separater
locale.setlocale(locale.LC_NUMERIC, "deu_DEU")

import numpy as np
import matplotlib.pyplot as plt
plt.rcdefaults()

# Tell matplotlib to use the locale we set above
plt.rcParams['axes.formatter.use_locale'] = True
%matplotlib qt5

figs, axes = plt.subplots(3,3, figsize=(9,6), sharex=True, sharey='row')

delays = []
locations = files_matrix_location
for row_id in range(len(locations)):
    for column_id in range(len(locations[0])):
        file = open(locations[row_id][column_id],'rb')
        data = pickle.load(file)
        data = data[1]['S-parameter']
        unwrapped = np.unwrap(np.angle(data[2])).T
        delays.append((unwrapped[-1,:]-unwrapped[0,:]+np.pi*2)/(data[1][1][-1]-data[1][1][0]))
delay = np.median(delays)



anticrossing_matrix = []
max_dev_matrix = []
coil_value_matrix = []

for row_id in range(len(locations)):
    anticrossing_row = []
    max_dev_row = []
    coil_value_row = []
    for column_id in range(len(locations[0])):
        file = open(locations[row_id][column_id],'rb')
        data = pickle.load(file)
        data = data[1]['S-parameter']
        data_nodelay = (data[2]*np.exp(-1j*delay*data[1][1])).T
        data_deviation_complex = data_nodelay - np.mean(data_nodelay, axis=0)
        axes[column_id, row_id].pcolormesh(data[1][0], data[1][1]/1e9, np.abs(data_deviation_complex), cmap='RdBu')
        max_dev = data[1][1][np.argmax(np.abs(data_deviation_complex), axis=0)]/1e9
        axes[column_id, row_id].plot(data[1][0], max_dev, color='yellow', linewidth=2)
        if column_id == 0:
            axes[column_id, row_id].set_title('Линия #'+str(row_id+1))
        if row_id == 0:
            axes[column_id, row_id].set_ylabel('Резонатор #'+str(column_id+1))
        anticrossing_row.append(data_deviation_complex)
        max_dev_row.append(max_dev)
        coil_value_row.append(data[1][0])
    anticrossing_matrix.append(anticrossing_row)
    max_dev_matrix.append(max_dev_row)
    coil_value_matrix.append(coil_value_row)
figs.text(0.5, 0.04, 'Напряжение управляющей линии, В', ha='center')
figs.text(0.02, 0.5, 'Частота, ГГц', va='center', rotation='vertical')
#plt.tight_layout()
print (delay/2/np.pi)

-2.3791658834395274e-08


In [51]:
fqb = lambda x, EJ1, EJ2, EC, phi0, L: (8*EC)**0.5*((EJ1-EJ2)**2*np.sin(np.pi*x*L+phi0*np.pi)**2+(EJ1+EJ2)**2*np.cos(np.pi*x*L+phi0*np.pi)**2)**0.25
fr  = lambda x, frb, g, EJ1, EJ2, EC, phi0, L: (fqb(x, EJ1, EJ2, EC, phi0, L)+frb)*0.5+(((fqb(x, EJ1, EJ2, EC, phi0, L)-frb)*0.5)**2+g**2)**0.5*np.sign(frb-fqb(x, EJ1, EJ2, EC, phi0, L))

In [52]:
num_resonators=3

In [53]:
L_mat=np.asarray([[ 0.31*8, -0.07*8+0.31*8, -0.02*8+0.31*8],
        [ 0.08*8+0.29*8,  0.29*8, -0.13*8+0.29*8],
        [-0.08*8+0.14*8,  0.07*8+0.14*8,  0.14*8]])
phi0 = np.asarray([-0.02, -0.025, -0.038])

initial_guess = {'L':L_mat,#np.identity(num_resonators)*0.3-0.01, # initial guess for inductance matrix
                     'phi0':phi0,#-np.ones(num_resonators)*0.1, # initial guess for residual flux
                     'frb':np.mean(max_dev_matrix, axis=(0,2))*1e9,
                     #'frb':[6.85e9, 7.15e9, 7.00e9],
                     'EC':0.175e9,
                     'EJ1':18.8e9,
                     'EJ2':3.6e9,
                     'g':40e6}

phi0_vec = []
L_mat = []
fitresults_vec = []
for resonator_id in range(num_resonators):
    data = [max_dev_matrix[i][resonator_id] for i in range(num_resonators)]
    x = [coil_value_matrix[i][resonator_id] for i in range(num_resonators)]
    def fr_coil(p, coil_id):
        frb, g, EJ1, EJ2, EC, phi0 = p[:6]
        L = p[6:]
        return fr(x[coil_id], frb, g, EJ1, EJ2, EC, phi0, L[coil_id])/1e9
    def residuals(p):
        frb, g, EJ1, EJ2, EC, phi0 = p[:6]
        L = p[6:]
        res = []
        for coil_id in range(len(data)):
            #print (fr(x[coil_id], frb, g, EJ1, EJ2, EC, phi0, L[coil_id]))
            #print (fqb(x[coil_id], EJ1, EJ2, EC, phi0, L[coil_id]))
            #print (x[coil_id]*np.reshape(L,(-1,1)))
            res.extend((fr(x[coil_id], frb, g, EJ1, EJ2, EC, phi0, L[coil_id])/1e9-data[coil_id])*(3 if coil_id == resonator_id else 1))
        return res
    from scipy.optimize import least_squares

    p0 = (initial_guess['frb'][resonator_id], 
          initial_guess['g'], 
          initial_guess['EJ1'], 
          initial_guess['EJ2'], 
          initial_guess['EC'], 
          initial_guess['phi0'][resonator_id])+\
          tuple(initial_guess['L'][resonator_id,:])

    bounds = ((0,      0,      0,      0,      0,      -np.pi)+tuple([-np.inf]*num_resonators),
              (np.inf, np.inf, np.inf, np.inf, np.inf,  np.pi)+tuple([ np.inf]*num_resonators))
    fitresults = least_squares(residuals, p0, x_scale=np.abs(p0), bounds=bounds)
    fitresults_vec.append(fitresults)
    phi0_vec.append(fitresults.x[5])
    L_mat.append(fitresults.x[6:])
    #print (fitresults.x)

In [54]:
for resonator_id in range(num_resonators):
    for coil_id in range(num_resonators):
        axes[resonator_id, coil_id].plot(coil_value_matrix[coil_id][resonator_id], 
                                     fr_coil(fitresults_vec[resonator_id].x, coil_id), color='black', linewidth=2)

In [46]:
np.asarray(L_mat), phi0_vec

(array([[ 2.4316565 ,  1.95215324,  2.29014836],
        [ 2.71281861,  2.17353468,  1.0736594 ],
        [-0.26116851,  1.46202884,  0.96886678]]),
 [-0.0005042147146843105, -0.0023648127926861324, -0.0007567410056142773])

In [48]:
for row_id in range(len(L_mat)):
    for col_id in range(len(L_mat)):
        if row_id != col_id:
            L_mat[row_id][col_id]-=L_mat[row_id][row_id]

In [49]:
L_mat

[array([ 2.4316565 , -0.47950326, -0.14150814]),
 array([ 0.53928393,  2.17353468, -1.09987528]),
 array([-1.23003529,  0.49316206,  0.96886678])]

In [55]:
np.dot(np.linalg.inv(L_mat), phi0_vec)

array([-0.00025544, -0.00137219,  0.00122073])

## Function to find the resonator for two-tone scans

In [22]:
def find_resonator(resonator_id, criterion='max_dev_complex', delay=0, check=1, raise_on_error=True): 
#    print ('resonator_id: ', resonator_id)
    # turn of all second tones
    lo1_status = lo1.get_status()
    lo1.set_status(0)
    # prior bandwidth
    bandwidth = pna.get_bandwidth()
    pna.set_bandwidth(500)
    average = pna.get_average()
    pna.set_average(0)
    #pna.set_power(-60)
    #pna.set_nop(501)
    nop = pna.get_nop()
    xlim = pna.get_xlim()
    pna.set_nop((qubits[resonator_id]['r']['Fr_max']-
                 qubits[resonator_id]['r']['Fr_min'])/qubits[resonator_id]['r']['dFr']+1)
    pna.set_xlim(qubits[resonator_id]['r']['Fr_min'], qubits[resonator_id]['r']['Fr_max'])
    #current_src1.set_current(current)
    
    # measure S21
    results = []
    for iteration_id in range(check+1):
        S21 = pna.measure()['S-parameter']
        freqs = pna.get_points()['S-parameter'][0][1]
    
        if (criterion == 'max_dev_complex'):
            measurement = S21*np.exp(2*np.pi*1j*freqs*delay)
            #plt.figure()
            #plt.plot(freqs, np.real(measurement-np.mean(measurement)))
            #plt.plot(freqs, np.imag(measurement-np.mean(measurement)))
            #plt.plot(freqs, np.abs(measurement-np.mean(measurement)))
            results.append(freqs[np.argmax(np.abs(measurement-np.mean(measurement)))])
        if (criterion == 'min_abs'):
            measurement = np.abs(S21)
            results.append(freqs[np.argmin(measurement)])
    # if resonator is off by more 3 times tolerance, raise if raise_on_error, otherwise warn
    if (np.std(results)>qubits[resonator_id]['r']['dFr']*10):
        if raise_on_error:
            raise Exception('Could not find resonator')
        else:
            print ('Could not find resonator')
    else: 
        print('resonator found at '+str(np.mean(results)))
        
    lo1.set_status(lo1_status)
    pna.set_bandwidth(bandwidth)
    pna.set_average(average)
    pna.set_nop(nop)
    pna.set_xlim(*xlim)
    
    return np.mean(results)

In [75]:
qubit_id='1'
delay = 70e-9
pna.pre_sweep()
res_freq = find_resonator(qubit_id, criterion='max_dev_complex', delay=delay, check=1, raise_on_error=False)
pna.post_sweep()

Could not find resonator


# Two-Tone Scan

In [79]:
voltage_max = 4
voltage_nop = 51
freq_nop = 701
nndac_channels = [6,8,10]#[0,2,4]

voltages=np.linspace(-.5,.5,voltage_nop)
#voltages=np.linspace(-voltage_max,voltage_max,voltage_nop)
freqs= np.linspace(4.4e9,5.3e9,freq_nop)
pna.set_nop(1)
pna.set_bandwidth(40)
pna.set_power(-20)
lo1.set_power(-20)
lo1.set_status(1)
delay = 70e-9
for qubit_id in ['1', '2', '3']:
    for coil_control_channel in [qubits[qubit_id]['q']['P']['nndac_coil_id']]:
#for nndac_channel_id in nndac_channels:
#    for qubit_id in ['1', '2', '3']:
        pna.set_power(qubits[qubit_id]['r']['P'])
        try: 
            coil = 'NNDAC-'+str(nndac_channel_id)
            #dac.set_voltage(0, channel=0)
            #dac.set_voltage(0, channel=2)
            #dac.set_voltage(0, channel=4)
            #dac.set_voltage(qubits[qubit_id]['q']['P']['Fmin_coil_currents'][6], channel=6)
            #dac.set_voltage(qubits[qubit_id]['q']['P']['Fmin_coil_currents'][8], channel=8)
            #dac.set_voltage(qubits[qubit_id]['q']['P']['Fmin_coil_currents'][10], channel=10)
            def set_volt_res(x):
                dac.set_voltage(x, channel=nndac_channel_id)
                res_freq = find_resonator(qubit_id, criterion='max_dev_complex', delay=delay,
                                          check=1, raise_on_error=False)
                pna.set_frequency(res_freq)
                pna.set_centerfreq(res_freq)
            result = sweep.sweep(pna,
                                 (voltages+qubits[qubit_id]['q']['P']['Fmin_coil_currents'][nndac_channel_id], 
                                  set_volt_res, 'Coil #{} voltage, qubit{}'.format(coil,qubit_id)),
                                 (freqs,lo1.set_frequency, 'Excitation frequency', 'Hz'),
                                 filename='Sample 1808_072-2 Coil #{} two-tone qubit{}'.format(coil,qubit_id))
            dac.set_voltage(0,nndac_channel_id)
        except KeyboardInterrupt:
            for nndac_channel in nndac_channels:
                dac.set_voltage(0,nndac_channel)
            raise
        finally:
            for nndac_channel in nndac_channels:
                dac.set_voltage(0,nndac_channel)

HTML(value='')

Started at:  Oct 12 2018 10:53:00
resonator found at 6600360000.0
resonator found at 6600320000.0
resonator found at 6600310000.0
resonator found at 6600230000.0
resonator found at 6600270000.0
resonator found at 6600180000.0
resonator found at 6600170000.0
resonator found at 6600160000.0
resonator found at 6600140000.0
resonator found at 6600140000.0
resonator found at 6600140000.0
resonator found at 6600060000.0
resonator found at 6600070000.0
resonator found at 6600040000.0
resonator found at 6599980000.0
resonator found at 6600030000.0
resonator found at 6600010000.0
resonator found at 6599960000.0
resonator found at 6599910000.0
resonator found at 6599940000.0
resonator found at 6599960000.0
resonator found at 6599930000.0
resonator found at 6599960000.0
resonator found at 6599980000.0
resonator found at 6599940000.0
resonator found at 6599950000.0
resonator found at 6599890000.0
resonator found at 6599930000.0
resonator found at 6599960000.0
resonator found at 6599970000.0
resona

HTML(value='')

Started at:  Oct 12 2018 10:58:29
resonator found at 6880690000.0
resonator found at 6880690000.0
resonator found at 6880590000.0
resonator found at 6880590000.0
resonator found at 6880590000.0
resonator found at 6880570000.0
resonator found at 6880540000.0
resonator found at 6880460000.0
resonator found at 6880440000.0
resonator found at 6880390000.0
resonator found at 6880350000.0
resonator found at 6880330000.0
resonator found at 6880260000.0
resonator found at 6880290000.0
resonator found at 6880270000.0
resonator found at 6880280000.0
resonator found at 6880240000.0
resonator found at 6880160000.0
resonator found at 6880120000.0
resonator found at 6880140000.0
resonator found at 6880180000.0
resonator found at 6880100000.0
resonator found at 6880070000.0
resonator found at 6880120000.0
resonator found at 6880030000.0
resonator found at 6880030000.0
resonator found at 6880030000.0
resonator found at 6879950000.0
resonator found at 6879920000.0
resonator found at 6879960000.0
resona

HTML(value='')

Started at:  Oct 12 2018 11:03:59
resonator found at 6735150000.0
resonator found at 6735200000.0
resonator found at 6735170000.0
resonator found at 6735130000.0
resonator found at 6735210000.0
resonator found at 6735140000.0
resonator found at 6735140000.0
resonator found at 6735100000.0
resonator found at 6735170000.0
resonator found at 6735160000.0
resonator found at 6735080000.0
resonator found at 6735170000.0
resonator found at 6735140000.0
resonator found at 6735060000.0
resonator found at 6735160000.0
resonator found at 6735080000.0
resonator found at 6735140000.0
resonator found at 6735110000.0
resonator found at 6735150000.0
resonator found at 6735150000.0
resonator found at 6735100000.0
resonator found at 6735100000.0
resonator found at 6735160000.0
resonator found at 6735070000.0
resonator found at 6735150000.0
resonator found at 6735080000.0
resonator found at 6735110000.0
resonator found at 6735060000.0
resonator found at 6735060000.0
resonator found at 6735110000.0
resona

# Inductance matrix

In [55]:
import qsweepy.save_pkl

## visualization of anticrossing

In [107]:
# plot all anitcrossings on one axes
figs, axes = plt.subplots(3,3, figsize=(12,9))
locations = [[r'D:\data\2018-09-19\17-35-56-Sample 1 Coil #NNDAC-0 anticrossings qubit 1', 
              r'D:\data\2018-09-19\17-38-23-Sample 1 Coil #NNDAC-0 anticrossings qubit 2',
              r'D:\data\2018-09-19\17-43-49-Sample 1 Coil #NNDAC-0 anticrossings qubit 3'],
            [r'D:\data\2018-09-19\17-48-29-Sample 1 Coil #NNDAC-2 anticrossings qubit 1',
             r'D:\data\2018-09-19\17-50-56-Sample 1 Coil #NNDAC-2 anticrossings qubit 2',
             r'D:\data\2018-09-19\17-56-23-Sample 1 Coil #NNDAC-2 anticrossings qubit 3'],
            [r'D:\data\2018-09-19\18-01-03-Sample 1 Coil #NNDAC-4 anticrossings qubit 1',
             r'D:\data\2018-09-19\18-03-30-Sample 1 Coil #NNDAC-4 anticrossings qubit 2',
             r'D:\data\2018-09-19\18-08-56-Sample 1 Coil #NNDAC-4 anticrossings qubit 3']]
locations = [[r'D:\data\2018-09-25\10-00-51-1808_072-2 Coil #NNDAC-18 anticrossings qubit 1',
             r'D:\data\2018-09-25\10-03-28-1808_072-2 Coil #NNDAC-18 anticrossings qubit 2',
             r'D:\data\2018-09-25\10-07-19-1808_072-2 Coil #NNDAC-18 anticrossings qubit 3'],
             [r'D:\data\2018-09-25\10-09-55-1808_072-2 Coil #NNDAC-20 anticrossings qubit 1', 
              r'D:\data\2018-09-25\10-12-35-1808_072-2 Coil #NNDAC-20 anticrossings qubit 2',
              r'D:\data\2018-09-25\10-16-25-1808_072-2 Coil #NNDAC-20 anticrossings qubit 3'],
            [r'D:\data\2018-09-25\10-19-05-1808_072-2 Coil #NNDAC-16 anticrossings qubit 1',
             r'D:\data\2018-09-25\10-21-43-1808_072-2 Coil #NNDAC-16 anticrossings qubit 2',
             r'D:\data\2018-09-25\10-25-33-1808_072-2 Coil #NNDAC-16 anticrossings qubit 3']]
#files = [['Sample 1 Coil #NNDAC-{0} anticrossings qubit {1}'.format(i,j) for j in ['1', '2', '3'] ] for i in [0,2,4]]
files = [['1808_072-2 Coil #NNDAC-{0} anticrossings qubit {1}'.format(i,j) for j in ['1', '2', '3']]
        for i in [18, 20, 16]]
delays = []
for row_id in range(len(locations)):
    for column_id in range(len(locations[0])):
        data = qsweepy.save_pkl.load_pkl(files[row_id][column_id],locations[row_id][column_id])[1]['S-parameter']
        unwrapped = np.unwrap(np.angle(data[2])).T
        delays.append((unwrapped[-1,:]-unwrapped[0,:]+2*np.pi)/(data[1][1][-1]-data[1][1][0]))
        print (np.median((unwrapped[-1,:]-unwrapped[0,:]+2*np.pi)))
delay = np.median(delays)

anticrossing_matrix = []

for row_id in range(len(locations)):
    anticrossing_row = []
    for column_id in range(len(locations[0])):
        data = qsweepy.save_pkl.load_pkl(files[row_id][column_id],locations[row_id][column_id])[1]['S-parameter']
        #unwrapped = np.unwrap(np.angle(data[2])).T
        #phase_diff = unwrapped[-1,:]-unwrapped[0,:]
        #data_nodelay = (data[2]*np.exp(-1j*np.linspace(0,np.median(phase_diff),unwrapped.shape[0]))).T
        data_nodelay = (data[2]*np.exp(-1j*delay*data[1][1])).T
        data_deviation_complex = data_nodelay - np.mean(data_nodelay, axis=0)
        anticrossing_row.append(data_deviation_complex)
        #axes[row_id,column_id].pcolormesh(data[1][0], data[1][1], np.angle(data_nodelay), cmap='RdBu')
        im = axes[row_id,column_id].pcolormesh(data[1][0], data[1][1], np.abs(data_deviation_complex), cmap='RdBu_r')
        plt.colorbar(im, ax=axes[row_id, column_id])
    anticrossing_matrix.append(anticrossing_row)
plt.tight_layout()
print (delay/2/np.pi)

D:\data\2018-09-25\10-00-51-1808_072-2 Coil #NNDAC-18 anticrossings qubit 1/1808_072-2 Coil #NNDAC-18 anticrossings qubit 1.pkl
-1.0196176348
D:\data\2018-09-25\10-03-28-1808_072-2 Coil #NNDAC-18 anticrossings qubit 2/1808_072-2 Coil #NNDAC-18 anticrossings qubit 2.pkl
-2.22031254233
D:\data\2018-09-25\10-07-19-1808_072-2 Coil #NNDAC-18 anticrossings qubit 3/1808_072-2 Coil #NNDAC-18 anticrossings qubit 3.pkl
-0.894897177689
D:\data\2018-09-25\10-09-55-1808_072-2 Coil #NNDAC-20 anticrossings qubit 1/1808_072-2 Coil #NNDAC-20 anticrossings qubit 1.pkl
-1.03462142961
D:\data\2018-09-25\10-12-35-1808_072-2 Coil #NNDAC-20 anticrossings qubit 2/1808_072-2 Coil #NNDAC-20 anticrossings qubit 2.pkl
-2.230807924
D:\data\2018-09-25\10-16-25-1808_072-2 Coil #NNDAC-20 anticrossings qubit 3/1808_072-2 Coil #NNDAC-20 anticrossings qubit 3.pkl
-0.762598594434
D:\data\2018-09-25\10-19-05-1808_072-2 Coil #NNDAC-16 anticrossings qubit 1/1808_072-2 Coil #NNDAC-16 anticrossings qubit 1.pkl
5.01619560885
D

In [96]:
plt.figure()
centre_pix, period_pix = fitting.xcorr_centre_period(anticrossing_matrix[1][1], axis=1, drop_size=0)

#centre_par = resonator_reference_spectrum['S-parameter'][1][0][int(centre_reference_pixels)], 
#period_par = (resonator_reference_spectrum['S-parameter'][1][0][1]-resonator_reference_spectrum['S-parameter'][1][0][0])*period_reference_pixels
print (centre_pix, period_pix)
len(data[1][0]), data[1][0]

42.5883575884 1


(201, array([-4.  , -3.96, -3.92, -3.88, -3.84, -3.8 , -3.76, -3.72, -3.68,
        -3.64, -3.6 , -3.56, -3.52, -3.48, -3.44, -3.4 , -3.36, -3.32,
        -3.28, -3.24, -3.2 , -3.16, -3.12, -3.08, -3.04, -3.  , -2.96,
        -2.92, -2.88, -2.84, -2.8 , -2.76, -2.72, -2.68, -2.64, -2.6 ,
        -2.56, -2.52, -2.48, -2.44, -2.4 , -2.36, -2.32, -2.28, -2.24,
        -2.2 , -2.16, -2.12, -2.08, -2.04, -2.  , -1.96, -1.92, -1.88,
        -1.84, -1.8 , -1.76, -1.72, -1.68, -1.64, -1.6 , -1.56, -1.52,
        -1.48, -1.44, -1.4 , -1.36, -1.32, -1.28, -1.24, -1.2 , -1.16,
        -1.12, -1.08, -1.04, -1.  , -0.96, -0.92, -0.88, -0.84, -0.8 ,
        -0.76, -0.72, -0.68, -0.64, -0.6 , -0.56, -0.52, -0.48, -0.44,
        -0.4 , -0.36, -0.32, -0.28, -0.24, -0.2 , -0.16, -0.12, -0.08,
        -0.04,  0.  ,  0.04,  0.08,  0.12,  0.16,  0.2 ,  0.24,  0.28,
         0.32,  0.36,  0.4 ,  0.44,  0.48,  0.52,  0.56,  0.6 ,  0.64,
         0.68,  0.72,  0.76,  0.8 ,  0.84,  0.88,  0.92,  0.96,  1.  ,
 

In [87]:
anticrossing_matrix[0][0].shape

(81, 201)

# Nice Q1-Q2 anticrossing for demonstration

## Qubit-resonator anticrossings at non-zero bias on second control

In [111]:
#coil = 'z1'
#current_max = 1.5e-3
voltage_max = 4
current_nop = 101
nndac_channels = [2]#[21,1,7]
#currents=np.linspace(-current_max,current_max,current_nop)
voltages=np.linspace(-voltage_max,voltage_max,current_nop)
#current_src1.set_status(1)
pna.set_nop(501)
pna.set_bandwidth(500)
pna.set_power(qubits[qubit_id]['r']['P']) 
coil2_voltage = 1.5
for nndac_channel_id in nndac_channels:
    for qubit_id in ['2']:
        dac.set_voltage(coil2_voltage, channel=0)
        coil = 'NNDAC-'+str(nndac_channel_id)
        #pna.set_xlim(qubits[qubit_id]['r']['Fr']-50e6,qubits[qubit_id]['r']['Fr']+50e6)
        pna.set_nop((qubits[qubit_id]['r']['Fr_max']-qubits[qubit_id]['r']['Fr_min'])/qubits[qubit_id]['r']['dFr']+1)
        pna.set_xlim(qubits[qubit_id]['r']['Fr_min'], qubits[qubit_id]['r']['Fr_max'])
        result = sweep.sweep(pna, 
                        (voltages, lambda x: dac.set_voltage(x, channel=nndac_channel_id), 'Coil #{} voltage, qubit{}'.format(coil,qubit_id)), 
                        filename='1809-001_2 Coil #{} anticrossings qubit {}'.format(coil, qubit_id))
        dac.set_voltage(0, channel=nndac_channel_id)

A Jupyter Widget

Started at:  Sep 20 2018 12:00:30
Exiting sweep main thread #{} via return 8744

Elapsed time: 0 h 4 m 9.0 s


## Qubit-resonator anticrossing at abstract control parameter for high-quality anticrossing picture

In [114]:
#coil = 'z1'
#current_max = 1.5e-3
voltage_max = 2.5
current_nop = 101
nndac_channels = [2]#[21,1,7]
#currents=np.linspace(-current_max,current_max,current_nop)
voltages=np.linspace(-voltage_max,voltage_max,current_nop)
#current_src1.set_status(1)
pna.set_nop(501)
pna.set_bandwidth(500)
pna.set_power(qubits[qubit_id]['r']['P']) 
coil2_voltage = 1.5
for nndac_channel_id in nndac_channels:
    for qubit_id in ['2']:
        dac.set_voltage(coil2_voltage, channel=0)
        coil = 'NNDAC-'+str(nndac_channel_id)
        #pna.set_xlim(qubits[qubit_id]['r']['Fr']-50e6,qubits[qubit_id]['r']['Fr']+50e6)
        pna.set_nop((qubits[qubit_id]['r']['Fr_max']-qubits[qubit_id]['r']['Fr_min'])/qubits[qubit_id]['r']['dFr']+1)
        pna.set_xlim(qubits[qubit_id]['r']['Fr_min'], qubits[qubit_id]['r']['Fr_max'])
        def set_sweep_parameter(V):
            dac.set_voltage(1.5-V, channel=0)
            dac.set_voltage(1.5+V, channel=2)
        result = sweep.sweep(pna, 
                        (voltages, set_sweep_parameter, 'Abstract voltage-like sweep parameter'.format(coil,qubit_id)), 
                        filename='1809-001_2 Coil #{} anticrossings qubit {}'.format(coil, qubit_id))
        dac.set_voltage(0, channel=nndac_channel_id)

A Jupyter Widget

Started at:  Sep 20 2018 12:16:13
Exiting sweep main thread #{} via return 11612

Elapsed time: 0 h 4 m 11.24 s


## Qubit-qubit anticrossing vs abstract control parameter

In [8]:
#L_mat=np.asarray([[ 0.31045445, -0.06621955, -0.02083502],
#        [ 0.08090813,  0.29212717, -0.13412727],
#        [-0.07698924,  0.06840561,  0.13873278]])

#phi0_vec = [-0.029960100879772234, -0.025342234038425226, -0.038382257248394662]


L_mat = np.asarray([[ 2.432, -0.476,  -0.144], [ 0.583 ,  2.178, -1.084], [-0.689,  0.473,  0.989  ]])
phi0_vec = np.asarray([-0.002, -0.004, 0.002])
np.asarray(L_mat), phi0_vec


(array([[ 2.432, -0.476, -0.144],
        [ 0.583,  2.178, -1.084],
        [-0.689,  0.473,  0.989]]), array([-0.002, -0.004,  0.002]))

In [23]:
voltage_max = 4
voltage_nop = 151
freq_nop = 301
nndac_channels = [6]
fluxes=np.linspace(0,0.5,voltage_nop)
#voltages=np.linspace(-voltage_max,voltage_max,voltage_nop)
freqs= np.linspace(4.85e9,6.05e9,freq_nop)
pna.set_nop(1)
pna.set_bandwidth(50)
lo1.set_power(-12)
pna.set_power(-20)
lo1.set_status(1)
delay = 50e-9
for nndac_channel_id in nndac_channels:
    for qubit_id in ['1', '2']:
        for nndac_channel in [6,8,10]:
            dac.set_voltage(0,nndac_channel)
        #pna.set_power(qubits[qubit_id]['r']['P'])
        try: 
            coil = 'NNDAC-'+str(nndac_channel_id)
            def set_flux_12(x):
                fluxes = [-0.5+x, -x, 0]
                voltages = np.linalg.solve(np.asarray(L_mat), -np.asarray(phi0_vec)+fluxes)
                dac.set_voltage(voltages[0], channel=6)
                dac.set_voltage(voltages[1], channel=8)
                dac.set_voltage(voltages[2], channel=10)
                #dac.set_voltage(1.7-V, channel=6)
                #dac.set_voltage(0.5+V, channel=8)
                #dac.set_voltage(x, channel=nndac_channel_id)
                res_freq = find_resonator(qubit_id, criterion='max_dev_complex', delay=delay, check=1, raise_on_error=False)
                pna.set_frequency(res_freq)
                pna.set_centerfreq(res_freq)
            result = sweep.sweep(pna,
                                 (fluxes, set_flux_12, 'Flux'.format(coil,qubit_id)),
                                 (freqs,lo1.set_frequency, 'Excitation frequency', 'Hz'),
                                 filename='Sample 1808-072_2 Coil #{} two-tone qubit{}'.format(coil,qubit_id))
            dac.set_voltage(0,nndac_channel_id)
        except KeyboardInterrupt:
            for nndac_channel in [6,8,10]:
                dac.set_voltage(0,nndac_channel)
            raise
        finally:
            for nndac_channel in [6,8,10]:
                dac.set_voltage(0,nndac_channel)
                


C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)


NameError: name 'dac' is not defined

In [26]:
voltage_max = 4
voltage_nop = 101
freq_nop = 101
#nndac_channels = [6]
fluxes=np.linspace(-0.75,0.25,voltage_nop)
#voltages=np.linspace(-voltage_max,voltage_max,voltage_nop)
freqs= np.linspace(4.75e9,5.95e9,freq_nop)
pna.set_nop(1)
pna.set_bandwidth(50)
lo1.set_power(-20)
pna.set_power(-50)
lo1.set_status(1)
delay = 50e-9
#for nndac_channel_id in nndac_channels:
for qubit_id, qubit_name in enumerate(['1', '2', '3']):
    try: 
        for coil_control_channel  in coil_control_channels:
            coil_device.set_output(1, channel=coil_control_channel)
        def set_qubit_flux(x):
            fluxes = [0 ,0, 0]
            fluxes[qubit_id] = x
            voltages = np.linalg.solve(np.asarray(L_mat), -np.asarray(phi0_vec)+fluxes)
            for channel_id, channel_voltage in enumerate(voltages):
                coil_device.set_offset(voltages[channel_id], channel=coil_control_channels[channel_id])
            res_freq = find_resonator(qubit_name, criterion='max_dev_complex', delay=delay, check=1, raise_on_error=False)
            pna.set_frequency(res_freq)
            pna.set_centerfreq(res_freq)
        result = sweep.sweep(pna,
                                 (fluxes, set_qubit_flux, 'Qubit {} SQUID flux'.format(qubit_name), 'Phi_0'),
                                 (freqs,lo1.set_frequency, 'Excitation frequency', 'Hz'),
                                 filename=sample_name+' two-tone qubit{}'.format(qubit_name))
    finally:
        for coil_control_channel  in coil_control_channels:
            coil_device.set_offset(0, channel=coil_control_channel)
            coil_device.set_output(0, channel=coil_control_channel)
                


C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)


HTML(value='')

Started at:  Oct 23 2018 11:00:43


C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)


resonator found at 6599920000.0
resonator found at 6599880000.0
resonator found at 6599820000.0
resonator found at 6599710000.0
resonator found at 6599700000.0
resonator found at 6599610000.0
resonator found at 6599570000.0
resonator found at 6599510000.0
resonator found at 6599450000.0
resonator found at 6599420000.0
resonator found at 6599370000.0
resonator found at 6599320000.0
resonator found at 6599270000.0
resonator found at 6599250000.0
resonator found at 6599220000.0
resonator found at 6599180000.0
resonator found at 6599140000.0
resonator found at 6599070000.0
resonator found at 6599070000.0
resonator found at 6599100000.0
resonator found at 6599070000.0
resonator found at 6599000000.0
resonator found at 6599030000.0
resonator found at 6599050000.0
resonator found at 6599000000.0
resonator found at 6599000000.0
resonator found at 6599030000.0
resonator found at 6599080000.0
resonator found at 6599050000.0
resonator found at 6599040000.0
resonator found at 6599050000.0
resonato

...

resonator found at 6879510000.0
resonator found at 6879470000.0
resonator found at 6879390000.0
resonator found at 6879370000.0
resonator found at 6879350000.0
resonator found at 6879190000.0
resonator found at 6879160000.0
resonator found at 6879170000.0
resonator found at 6879130000.0
resonator found at 6878990000.0
resonator found at 6878990000.0
resonator found at 6879030000.0
resonator found at 6879010000.0
resonator found at 6878920000.0
resonator found at 6878880000.0
resonator found at 6878920000.0
resonator found at 6878810000.0
resonator found at 6878850000.0
resonator found at 6878810000.0
resonator found at 6878760000.0
resonator found at 6878790000.0
resonator found at 6878740000.0
resonator found at 6878720000.0
resonator found at 6878730000.0
resonator found at 6878760000.0
resonator found at 6878700000.0
resonator found at 6878770000.0
resonator found at 6878750000.0
resonator found at 6878720000.0
resonator found at 6878710000.0
resonator found at 6878720000.0
resonato

HTML(value='')

Started at:  Oct 23 2018 11:22:30


C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)


resonator found at 6734550000.0
resonator found at 6734570000.0
resonator found at 6734430000.0
resonator found at 6734370000.0
resonator found at 6734370000.0
resonator found at 6734240000.0
resonator found at 6734310000.0
resonator found at 6734200000.0
resonator found at 6734160000.0
resonator found at 6734140000.0
resonator found at 6734090000.0
resonator found at 6734080000.0
resonator found at 6733950000.0
resonator found at 6734060000.0
resonator found at 6734020000.0
resonator found at 6733950000.0
resonator found at 6733820000.0
resonator found at 6733850000.0
resonator found at 6733800000.0
resonator found at 6733890000.0
resonator found at 6733870000.0
resonator found at 6733930000.0
resonator found at 6733830000.0
resonator found at 6733840000.0
resonator found at 6733880000.0
resonator found at 6733820000.0
resonator found at 6733840000.0
resonator found at 6733800000.0
resonator found at 6733840000.0
resonator found at 6733890000.0
resonator found at 6733900000.0
resonato

In [40]:
coil_device

In [69]:
pna.pre_sweep()
plt.plot(pna.get_freqpoints(),20*np.log10(np.abs(pna.measure()['Voltage'])))
pna.post_sweep()

KeyError: 'Voltage'

## Simple two-tone

In [18]:
voltage_max = 4
voltage_nop = 41
freq_nop = 401
nndac_channels = [2]
voltages=np.linspace(1,2.7,voltage_nop)
#voltages=np.linspace(-voltage_max,voltage_max,voltage_nop)
freqs= np.linspace(5e9,6.2e9,freq_nop)
pna.set_nop(1)
pna.set_bandwidth(50)
lo1.set_power(-0)
lo1.set_status(1)
delay = 55e-9
for nndac_channel_id in nndac_channels:
    for qubit_id in ['2']:
        pna.set_power(qubits[qubit_id]['r']['P'])
        try: 
            coil = 'NNDAC-'+str(nndac_channel_id)
            def set_volt_res(x):
                dac.set_voltage(x, channel=nndac_channel_id)
                res_freq = find_resonator(qubit_id, criterion='min_abs', delay=delay, check=1, raise_on_error=False)
                pna.set_frequency(res_freq)
                pna.set_centerfreq(res_freq)
            result = sweep.sweep(pna,
                                 (voltages, set_volt_res, 'Coil #{} voltage, qubit{}'.format(coil,qubit_id)),
                                 (freqs,lo1.set_frequency, 'Excitation frequency', 'Hz'),
                                 filename='Sample 809-001_2 Coil #{} two-tone qubit{}'.format(coil,qubit_id))
            dac.set_voltage(0,nndac_channel_id)
        except KeyboardInterrupt:
            for nndac_channel in nndac_channels:
                dac.set_voltage(0,nndac_channel)
            raise
        finally:
            for nndac_channel in nndac_channels:
                dac.set_voltage(0,nndac_channel)
                


A Jupyter Widget

Started at:  Sep 19 2018 19:21:26
resonator found at 6892100000.0
resonator found at 6890600000.0
resonator found at 6889300000.0
resonator found at 6888000000.0
resonator found at 6887000000.0
resonator found at 6886300000.0
resonator found at 6885500000.0
resonator found at 6885100000.0
resonator found at 6884600000.0
resonator found at 6884000000.0
resonator found at 6883500000.0
resonator found at 6883100000.0
resonator found at 6882900000.0
resonator found at 6882500000.0
resonator found at 6882300000.0
resonator found at 6882200000.0
resonator found at 6881800000.0
resonator found at 6881800000.0
resonator found at 6881700000.0
resonator found at 6881400000.0
resonator found at 6881600000.0
resonator found at 6881200000.0
resonator found at 6881200000.0
resonator found at 6881100000.0
resonator found at 6881100000.0
resonator found at 6881200000.0
resonator found at 6880900000.0
resonator found at 6881000000.0
resonator found at 6881400000.0
resonator found at 6881300000.0
resona

## Кубитная спектроскопия

# Импульсы

## Неработающая Оффлайн-калибровка импульсов

In [8]:
power1 = 10**(sa.measure()['Power']/10)*1e-3
power1

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)


array([1.72993986e-12, 1.58038637e-12, 1.55080739e-12, ...,
       1.91789447e-12, 1.84296320e-12, 1.93404341e-12])

In [57]:
import tqdm
sa.set_bandwidth(0.5e6)
sa.set_nop(1001)
sequence_length = 9600
pause_length = 4700
signal_length = sequence_length-2*pause_length
shot_noise_time=30

sequences1 = []
sequences2 = []
measurement_matrix_sequence = [] #amplitudes on I are axis 0, powers on Q are axis 1
num_averages = 20 # for averaging
amplitude_list = [0.0, -0.1, 0.1, 1.0]

for random_sequence_id in tqdm.tqdm(range(20)):
    dac_sequence1 = []
    dac_sequence2 = []
    while len(dac_sequence1) < sequence_length:
        dac_sequence1.extend([1]*np.random.randint(shot_noise_time)+[-1]*np.random.randint(shot_noise_time))
    while len(dac_sequence2) < sequence_length:
        dac_sequence2.extend([1]*np.random.randint(shot_noise_time)+[-1]*np.random.randint(shot_noise_time))
    dac_sequence1 = np.asarray(dac_sequence1[:sequence_length])
    dac_sequence2 = np.asarray(dac_sequence2[:sequence_length])
    dac_sequence1[-pause_length:] = 0
    dac_sequence2[-pause_length:] = 0
    dac_sequence1[:pause_length] = 0
    dac_sequence2[:pause_length] = 0
    
    sequences1.append(dac_sequence1)
    sequences2.append(dac_sequence2)
    
    
    awg1.stop()
    awg2.stop()
    
    awg1.set_waveform(dac_sequence1*0.0, channel=0)
    awg1.set_waveform(dac_sequence2*0.0, channel=1)
    awg2.set_amplitude(1.0, channel=0)
    awg2.set_amplitude(1.0, channel=1)
    measurement_matrix = []
    for amp_I_id, amp_I in enumerate(amplitude_list):
        measurement_row = []
        awg2.stop()
        awg2.set_waveform(dac_sequence1*amp_I, channel=0)
        for amp_Q_id, amp_Q in enumerate(amplitude_list):
            awg2.stop()
            awg2.set_waveform(dac_sequence2*amp_Q, channel=1)
            awg2.run()
            measurement_row.append(10**sa.measure()['Power']/10)
        measurement_matrix.append(measurement_row)
    measurement_matrix_sequence.append(measurement_matrix)

  0%|                                                                                           | 0/20 [00:00<?, ?it/s]C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)
100%|█████████████████████████████████████████████████████████████████████████████████| 20/20 [42:37<00:00, 129.74s/it]


In [75]:
awg_device.get_clock = lambda: 1e9

NameError: name 'seq_Q' is not defined

In [42]:
awg_master=awg2
master_channel=0
master_physical_delay = 400

def set_global_repetition_period(period):
    awg_master.stop()
    awg_master.set_waveform([0]*(period-master_physical_delay), channel=master_channel)
    awg_master.set_output(1, channel=master_channel)
    awg_master.run()

In [43]:
set_global_repetition_period(2000)

In [278]:
num_samples = 5
awg_device = awg2
sa_device = sa
lo_device = lo1
awg_channel_I = 2
awg_channel_Q = 3
calibration_sequence_length = 1600 
physical_delay = 400
calibration_sequence_margin = 600
random_sequence_num = 20
amplitudes = [0,0.2]
loss_theory = 1e-3

amplitude_small_id = 1
amplitude_zero_id = 0
#amplitude_negitive_id = 0
#amplitude_large_id = 4

sa.set_bandwidth(awg_device.get_clock()/repetition_period/2)
sa.set_bandwidth_video(10e3)
sa.set_detector('RMS')
repetition_period = calibration_sequence_length+physical_delay # in awg samples
sa.set_nop(repetition_period)
sa.set_centerfreq(lo_device.get_frequency()-awg_device.get_clock()/repetition_period/2.)
sa.set_span(awg_device.get_clock()*(repetition_period-1)/repetition_period)

# Perform background measurements
#c = np.asarray(measurement_matrix_sequence)
lo1.set_status(0)
incoherent_bg_samples = [10**(sa.measure()['Power']/10) for i in range(num_samples)]
incoherent_bg = np.mean(incoherent_bg_samples, axis=0)
incoherent_bg_error = np.std(incoherent_bg_samples)/np.sqrt(num_samples-2)
lo1.set_status(1)
awg_device.stop()
#awg_device.set_offset(0, channel=awg_channel_I)
#awg_device.set_offset(0, channel=awg_channel_Q)
awg_device.set_output(1, channel=awg_channel_I)
awg_device.set_output(1, channel=awg_channel_Q)
coherent_bg_samples = [10**(sa.measure()['Power']/10) for i in range(num_samples)]
coherent_bg = np.mean(coherent_bg_samples, axis=0)-incoherent_bg
coherent_bg_std = np.std(coherent_bg_samples)/np.sqrt(num_samples-2)

spectra = np.zeros((len(amplitudes), len(amplitudes), random_sequence_num, sa.get_nop()))
waveforms_I = np.zeros((random_sequence_num, repetition_period))
waveforms_Q = np.zeros((random_sequence_num, repetition_period))
awg_device.stop()
# measure random waveforms
for random_sequence_id in tqdm.tqdm(range(20)):
    seq_I = np.random.random(calibration_sequence_length)*2.-1.
    seq_Q = np.random.random(calibration_sequence_length)*2.-1.
    seq_I[:calibration_sequence_margin] = 0
    seq_I[-calibration_sequence_margin:]= 0
    seq_Q[:calibration_sequence_margin] = 0
    seq_Q[-calibration_sequence_margin:]= 0
    seq_I_full = np.asarray([0]*physical_delay+seq_I.tolist())
    seq_Q_full = np.asarray([0]*physical_delay+seq_Q.tolist())
    waveforms_I[random_sequence_id, :] = seq_I_full
    waveforms_Q[random_sequence_id, :] = seq_Q_full
    for amp_I_id, amp_I in enumerate(amplitudes):
        awg_device.set_waveform(seq_I*amplitudes[amp_I_id], channel=awg_channel_I)
        for amp_Q_id, amp_Q in enumerate(amplitudes):
            awg_device.set_waveform(seq_Q*amplitudes[amp_Q_id], channel=awg_channel_Q)
            awg_device.run()
            spectra[amp_I_id, amp_Q_id, random_sequence_id, :] = 10**(sa.measure()['Power']/10)
            awg_device.stop()
#plt.figure()
#plt.semilogy(incoherent_bg)
#plt.semilogy(coherent_bg)

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)







  0%|                                                                                           | 0/20 [00:00<?, ?it/s]






  5%|████▏                                                                              | 1/20 [00:03<01:15,  3.96s/it]






 10%|████████▎                                                                          | 2/20 [00:07<01:10,  3.94s/it]






 15%|████████████▍                                                                      | 3/20 [00:11<01:06,  3.93s/it]






 20%|████████████████▌                                                                  | 4/20 [00:15<01:02,  3.92s/it]






 25%|████████████████████▊                                                              | 5/20 [00:19<00:59,  3.94s/it]






 30%|████████████████████████▉

In [83]:
plt.figure()
plt.plot(seq_I)

In [84]:
awg_device.run()

In [90]:
amplitude_negative_id=amplitude_negitive_id

In [271]:
model_I = np.fft.fftshift(np.fft.fft(np.fft.fftshift(waveforms_I, axes=1), axis=1, norm='ortho'), axes=1) # in Watts
model_Q = np.fft.fftshift(np.fft.fft(np.fft.fftshift(waveforms_Q, axes=1), axis=1, norm='ortho'), axes=1) # in Watts
iq_phase_diff = np.exp(1j*np.angle(model_I*np.conj(model_Q)))
model_I_psd = np.abs(model_I)**2
model_Q_psd = np.abs(model_Q)**2
model_IQp_psd = np.abs(model_I+1j*model_Q)**2
model_IQn_psd = np.abs(model_I-1j*model_Q)**2

mean_model_I_psd = np.mean(model_I_psd, axis=0)
mean_model_Q_psd = np.mean(model_Q_psd, axis=0)

mean_measurement_I_psd = np.mean(spectra[amplitude_small_id, amplitude_zero_id, :, :], axis=0)
mean_measurement_Q_psd = np.mean(spectra[amplitude_zero_id, amplitude_small_id, :, :], axis=0)

mean_measurement_I_abs = np.mean(np.sqrt(spectra[amplitude_small_id, amplitude_zero_id, :, :]), axis=0)
mean_measurement_Q_abs = np.mean(np.sqrt(spectra[amplitude_zero_id, amplitude_small_id, :, :]), axis=0)

response_function_I_abs = np.sqrt(mean_measurement_I_psd/(mean_model_I_psd*amplitudes[amplitude_small_id]**2))
response_function_Q_abs = np.sqrt(mean_measurement_Q_psd/(mean_model_Q_psd*amplitudes[amplitude_small_id]**2))

interference_term = (spectra[amplitude_small_id, amplitude_small_id, :, :]-\
                     spectra[amplitude_small_id, amplitude_zero_id, :, :]-\
                     spectra[amplitude_zero_id, amplitude_small_id, :, :])
theory_interference_term = 2*np.sqrt(spectra[amplitude_zero_id, amplitude_small_id, :, :]\
                          *spectra[amplitude_small_id, amplitude_zero_id, :, :])
                          
interference_term_response = interference_term/theory_interference_term
interference_term_exp = (interference_term_response+1j*np.sqrt(1-interference_term_response**2))*iq_phase_diff
interference_term_cexp = (interference_term_response-1j*np.sqrt(1-interference_term_response**2))*iq_phase_diff
interference_term_response[np.abs(theory_interference_term)<np.percentile(np.abs(theory_interference_term), 5)]=np.nan
interference_term_exp[np.abs(theory_interference_term)<np.percentile(np.abs(theory_interference_term), 5)]=np.nan
interference_term_cexp[np.abs(theory_interference_term)<np.percentile(np.abs(theory_interference_term), 5)]=np.nan

interference_term_response_mean = np.nanmean(interference_term_response*iq_phase_diff, axis=0)
interference_term_exp_mean = np.nanmean(interference_term_response, axis=0)
interference_term_cexp_mean = np.nanmean(interference_term_response, axis=0)

sorted_phases = np.sort(np.angle(np.vstack([interference_term_cexp, interference_term_exp])), axis=0)
response_phase = []
for frequency_id in range(sorted_phases.shape[1]):
    points_current = sorted_phases[:, frequency_id]
    points_current = points_current[np.logical_not(np.isnan(points_current))].tolist()
    num_point_initial = len(points_current)
    for removed_point_id in range(int(num_point_initial/2)+2):
        if np.median(points_current)-points_current[0]>points_current[-1]-np.median(points_current):
            del points_current[0]
        else:
            del points_current[-1]
    response_phase.append(np.mean(points_current))
response_phase = np.asarray(response_phase)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in sqrt
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in sqrt


In [274]:
linear_model = np.abs(amplitudes[amplitude_small_id]*\
                      (model_I*response_function_I_abs+model_Q*response_function_Q_abs*np.exp(1j*response_phase)))**2
measurement = spectra[amplitude_small_id, amplitude_small_id, :, :]
plt.figure()
plt.semilogy(linear_model[0,:])
plt.semilogy(measurement[0,:])

In [276]:
plt.figure()
plt.plot(response_function_I_abs)
plt.plot(response_function_Q_abs)

In [277]:
plt.figure()
plt.plot(response_phase)

In [215]:
plt.figure()
plt.plot(np.nanmedian(np.angle(np.vstack([interference_term_cexp, interference_term_exp])), axis=0))

stat = np.angle(np.vstack([interference_term_cexp, interference_term_exp]))
stat[np.isnan(stat)]=0

In [230]:
hist = [np.histogram(stat[:, i], bins=np.linspace(-np.pi, np.pi, 40, endpoint=False))[0] for i in range(2000)]

In [228]:
[i.shape for i in hist]

[(19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),
 (19,),


In [231]:
plt.figure()
plt.pcolormesh(hist)

In [202]:
plt.figure()
plt.plot(interference_term_cexp[0,:])
plt.plot(interference_term_exp[0,:])

C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\numeric.py:492: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


In [200]:
plt.figure()
plt.plot((interference_term_response*iq_phase_diff).T)

C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\numeric.py:492: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


In [196]:
plt.figure()
plt.plot(np.angle(interference_term_response_mean))
plt.plot(np.angle(interference_term_exp_mean))
plt.plot(np.angle(interference_term_cexp_mean))
#plt.plot(np.angle(interference_term_response_mean2))

In [125]:
plt.figure()
plt.plot(pos_interference_noavg[0,:])
plt.plot(2*np.imag(model_I*response_function_I_abs*amplitudes[amplitude_small_id]*np.conj(model_Q*response_function_Q_abs)*amplitudes[amplitude_small_id])[0,:])

In [138]:
plt.figure()
plt.plot(interference_term[0,:])
plt.plot(theory_interference_term[0,:])

In [107]:
plt.figure()
plt.plot(np.unwrap(np.angle(model_I[1,:])))
plt.plot(np.unwrap(np.angle(model_Q[1,:])))

In [89]:
plt.figure()
plt.semilogy(model_I_psd[0, :])
plt.semilogy(spectra[amplitude_small_id, amplitude_zero_id, 0, :]*1e4)

In [78]:
plt.figure()
plt.plot(mean_model_I_psd)
plt.plot(mean_measurement_I_psd*1e4)


In [88]:
plt.figure()
plt.plot(response_function_I_abs)
plt.plot(response_function_Q_abs)

In [114]:
plt.figure()
plt.plot(np.unwrap(np.angle(iq_phase_diff[0,:]).T))

In [109]:
plt.figure()
plt.plot(pos_interference)
plt.plot(pos_interference_nophase)

C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\numeric.py:492: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


In [17]:
import pickle
with open(r'C:\Users\cryopxi\Documents\offline_calibration_data.pickle', 'wb') as f:
    pickle.dump({'sequences1':sequences1, 
                 'sequences2':sequences2, 
                 'measurements_low_power':measurements_low_power, 
                 'measurements_high_power':measurements_high_power, 
                 'freqpoints':sa.get_freqpoints(), 
                 'samplerate':awg2.get_clock()}, f)


### Загружаем импульсные скрипты всякие

In [9]:
import imp
imp.reload(awg_digital)
imp.reload(awg_iq_multi)
imp.reload(data_reduce)
imp.reload(fitting)
imp.reload(plotting)
imp.reload(tomography)
imp.reload(sweep)

<module 'qsweepy.sweep' from 'C:\\qtlab_replacement\\qsweepy\\sweep.py'>

In [10]:
# Источник тока - в autorange
#current.set_autorange(1)
#Мощности гетеродинов, постоянные
#Мощность гетеродина для возбуждения 13-16 дБм

lo_ex_pow = 14
lo_ex.set_status(1)
lo_ro_pow = 16
lo_ro.set_power(lo_ro_pow)

pna.write("OUTP ON")
pna.write("SOUR1:POW1:MODE ON")
pna.write("SOUR1:POW2:MODE OFF")
pna.set_sweep_mode("CW")
lo_ex.set_power(lo_ex_pow)

marker_length = 100
#readout_trigger_delay = 185
readout_trigger_delay = 0
trg_length = 10e-9

ex_clock = 1000e6
ro_clock = 1250e6

rep_rate = 20e3 # частота повторений эксперимента

#awg.stop()
awg1.check_cached=True
awg2.check_cached=True
#awg_tek.set_clock(ex_clock) # клок всех авгшк
awg1.set_clock(ex_clock)
awg2.set_clock(ex_clock)
awg1.set_clock_output(True)
awg2.set_clock_output(True)
awg1.set_nop(ex_clock/rep_rate) # репрейт нужно задавать по=хорошему только на управляющей,
awg2.set_nop(ex_clock/rep_rate) # репрейт нужно задавать по=хорошему только на управляющей,
awg1.run()
awg2.run()
# а вот длину сэмплов, которая очевидно то же самое, нужно задавать на всех авгшках.
# хорошо, что сейчас она только одна.
#this is zashkvar

# channel 0 is master and triggers all others
# M3202A settings (ignored on AWG5014C)
#awg._visainstrument.write('AWGC:DOUT1 ON')
#awg._visainstrument.write('AWGC:DOUT2 ON')
#awg._visainstrument.write('AWGC:DOUT3 ON')
#awg._visainstrument.write('AWGC:DOUT4 ON')
awg2.trigger_source_types = [0,6,6,6]
awg2.trigger_source_channels = [0, 4000,4000,4000]
awg2.trigger_delays = [30+10,7+10,7+10,7+10]
awg2.trigger_behaviours = [0,4,4,4]
awg2.set_trigger_mode(2)
awg1.set_trigger_mode(2)
awg1.trigger_source_types = [6,6,6,6]
awg1.trigger_source_channels = [4000, 4000,4000,4000]
awg1.trigger_delays = [7+10,7+10,7+10,7+10] # wait 60 ns for sync with awg2
awg1.trigger_behaviours = [4,4,4,4]
for channel in range(0,4):
    awg2.set_amplitude(1.0, channel=channel)
    awg2.set_offset(0.0,channel=channel)
    awg2.set_output(1, channel=channel)
    awg2.set_waveform(waveform = [0]*50000, channel=channel)
for channel in range(0,4):
    awg1.set_amplitude(1.0, channel=channel)
    awg1.set_offset(0.0,channel=channel)
    awg1.set_output(1, channel=channel)
    awg1.set_waveform(waveform =[0]*50000, channel=channel)
awg2.set_marker(length=100, delay=0, channel=0, pxi_channels=1)
awg1.set_marker(length=200, delay=0, channel=0, pxi_channels=2)
awg2.run()
awg1.run()

    
#awg.set_marker(length=marker_length, delay=0, channel=0) #M3202A


awg_channels =dict()
ro_trg = awg_digital.awg_digital(awg_tek, 1)
ro_trg.mode = 'set_delay' #M3202A
ro_trg.delay_setter = lambda x: adc.set_trigger_delay(int(x*adc.get_clock()/iq_ex.get_clock()-readout_trigger_delay)) #M3202A
#ro_trg.mode = 'waveform' #AWG5014C
awg_channels['ro_trg'] = ro_trg
# К первому маркере первого канала тектроникса подсоединён триггер оцифровщика.
# Надо подавать какой-нибудь триггер, что-ли.
# А ещё нужно подавать клок. К сожалению наш оцифровщик не может в 1ГГц клок, ему нужен 500МГц клок.
# Будем подавать 500 МГц с цифрового выхода тектроникса.
# AWG5014C
#awg_tek.set_marker1_low(-0.0, channel=1)
#awg_tek.set_marker2_low(-0.0, channel=1)
#awg_tek.set_marker1_high(1.0, channel=1)
#awg_tek.set_marker2_high(1.0, channel=1)
#awg_tek.set_marker1_low(-0.04, channel=2)
#awg_tek.set_marker2_low(-0.04, channel=2) 
#awg_tek.set_marker1_high(0.04, channel=2) # с клока хватит и такой амплитуды
#awg_tek.set_marker2_high(0.04, channel=2) # с клока хватит и такой амплитуды

# настройки оцифровщика
# Spectrum M3i-3212
#adc.reset()            
#adc.set_timeout(10000)
#adc.set_clock(ro_clock)

#adc.set_spc_samplerate(smp_rate)
#External trigger
adc.reset()
#Установить sample rate
adc.set_clock(ro_clock)
adc.set_clock_mode('pxi')
#adc.set_clock_mode('ext')
#adc.set_ext_clock(1000e6/80)
#adc.set_ext_clock(625e6/60)
#Выбрать канал (оба)
adc.select_channel01()
adc.set_channel01_amps(200,200)

adc.set_trigger_ext0_level0(0) # logical 0 value to 200 mV
adc.set_trigger_ext0_level1(2000) # logical 1 value to 600 mV
adc.trigger_mode_pos()
adc.set_trigger_ext0_pulsewidth(0)

adc.set_trigger_ext0()
adc.set_trigger_termination(1)
#Сегментированный режим
#adc.set_multi_mode()
#Настроить входнй диапазон в мв
#adc.init_channel01_multiple_recording()
#Настроить режим измерения
adc.set_multi_record_mode()

#Выбрать канал (оба)

In [9]:
awg_tek._visainstrument.write('AWGC:RMOD TRIG')
awg_tek._visainstrument.write('TRIG:WVAL LAST')
awg_tek._visainstrument.write('TRIG:IMP 50ohm')
awg_tek._visainstrument.write('SOUR1:ROSC:SOUR EXT')
awg_tek._visainstrument.write('SOUR1:ROSC:FREQ 10MHz')
awg_tek.set_clock(1e9)
awg_tek.set_nop(ex_clock/rep_rate-2000) # репрейт нужно задавать по=хорошему только на управляющей_t

True

In [10]:
j=200

In [12]:
j=j+2
awg1.stop()
awg2.stop()
for i in [0,1,2,3]:
    awg1.set_waveform(([1]*50+[0]*50)*j+[0]*10000, channel=i)
    awg2.set_waveform(([1]*50+[0]*50)*j+[0]*10000, channel=i)
    awg_tek.set_waveform(([1]*50+[0]*50)*j+[0]*8000, channel=i+1)
awg1.run()
awg2.run()
awg_tek.run()

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)


In [12]:
adc.get_clock()

1250000000

### Калибровка миксеров

In [41]:
#json.invalidate_calibrations()

In [42]:
#ro_if = 75e6
lo_freq = 5.15e9#5.7 e9
lo_ro_freq = 6.7e9
    
imp.reload(awg_iq_multi)
# промежуточные частоты для гетеродинной схемы new:
lo1.set_frequency(lo_freq)
lo_ro.set_frequency(lo_ro_freq)
iq_ex = awg_iq_multi.awg_iq_multi(awg_tek, awg_tek, 0, 1, lo_ex) #M3202A
iq_ro = awg_iq_multi.awg_iq_multi(awg1, awg1, 0, 1, lo_ro) #M3202A
#iq_ex = awg_iq_multi.awg_iq_multi(awg_tek, awg_tek, 1, 2, lo_ex) #M3202A
#iq_ro = awg_iq_multi.awg_iq_multi(awg_tek, awg_tek, 4, 3, lo_ro) #AWG5014C
readout_channels = {}
for _qubit_id, qubit in qubits.items():
    for tr,freq in qubits[_qubit_id]['q']['F'].items():
        iq_ex.carriers['iq_ex_q{}_'.format(_qubit_id)+tr] = awg_iq_multi.carrier(iq_ex)
        iq_ex.carriers['iq_ex_q{}_'.format(_qubit_id)+tr].set_frequency(freq)
        awg_channels['iq_ex_q{}_'.format(_qubit_id)+tr]=iq_ex.carriers['iq_ex_q{}_'.format(_qubit_id)+tr]
        iq_ro.carriers['iq_ro_q{}'.format(_qubit_id)] = awg_iq_multi.carrier(iq_ro)
        iq_ro.carriers['iq_ro_q{}'.format(_qubit_id)].set_frequency(qubits[_qubit_id]['r']['Fr'])
        awg_channels['iq_ro_q{}'.format(_qubit_id)] = iq_ro.carriers['iq_ro_q{}'.format(_qubit_id)]
        readout_channels['iq_ro_q{}'.format(_qubit_id)] = awg_channels['iq_ro_q{}'.format(_qubit_id)]

pg = pulses.pulses(awg_channels)
#print(ex_if*1e-6 ,lo_freq)

print("Excitation channel", end='\n')
iq_ex.do_calibration(sa)
print("Readout channel", end='\n')
iq_ro.do_calibration(sa)

Excitation channel
c:\data/calibrations/iq-dc/ lo_freq-5.15e+09
c:\data/calibrations/iq-rf/ frequency-4.985e+09-if--1.65e+08-sideband_id-     0
c:\data/calibrations/iq-rf/ frequency-5.01e+09-if--1.4e+08-sideband_id-     0
c:\data/calibrations/iq-rf/ frequency-4.833e+09-if--3.17e+08-sideband_id-     0
Readout channel
c:\data/calibrations/iq-dc/ lo_freq-6.7e+09
c:\data/calibrations/iq-rf/ frequency-6.6e+09-if--1e+08-sideband_id-     0
c:\data/calibrations/iq-rf/ frequency-6.88e+09-if-1.8e+08-sideband_id-     0
c:\data/calibrations/iq-rf/ frequency-6.735e+09-if-3.5e+07-sideband_id-     0


## задержка считывания

In [43]:
trigger_readout_seq = [pg.p('ro_trg', 10e-9, pg.rect, 1)]

adc.set_nums(50)
adc.set_nop(32768)
#adc.set_posttrigger(adc.get_nop()-4096)
if hasattr(adc, 'set_posttrigger'):
    adc.set_posttrigger(adc.get_nop()-32)
modem = modem_readout.modem_readout(pg, adc, trigger_readout_seq, axis_mean=0)
modem.readout_channels = readout_channels
#ro_trg.delay_setter = lambda x: adc.set_trigger_delay(int(x*adc.get_clock()/iq_ex.get_clock()-0)) # M3201A & M3202A
ro_trg.delay = 0#Spectrum+AWG5014C
delay_calibration = modem.calibrate_delay()
readout_trigger_delay = delay_calibration[awg_channels['iq_ro_q1']]
#readout_trigger_delay_cycles = readout_trigger_delay*adc.get_clock() M3201A
readout_trigger_delay_cycles = readout_trigger_delay*ro_trg.get_clock()
print (readout_trigger_delay, readout_trigger_delay_cycles)
ro_trg.delay_setter = lambda x: adc.set_trigger_delay(int(x*adc.get_clock()/iq_ex.get_clock())-readout_trigger_delay*adc.get_clock()) #M3201A
ro_trg.delay = int(-readout_trigger_delay_cycles)#Spectrum+AWG5014C
readout_trigger_delay_test = modem.calibrate_delay()[awg_channels['iq_ro_q1']]
#readout_trigger_delay_cycles_test = readout_trigger_delay_test*adc.get_clock() M3201A
readout_trigger_delay_cycles_test = readout_trigger_delay_test*ro_trg.get_clock()
print (readout_trigger_delay_test, readout_trigger_delay_cycles_test)

modem.calibrate_dc()

-4.84e-07 -484.00000000000006
2.4e-09 2.4


{'iq_ro_q1': {'iq_calibration': array([-0.4623986 +0.35349914j, -0.03461442+0.01502144j]),
  'coherent_background': array([-0.5 -0.2j , -0.28+0.34j, -0.06+0.5j , ..., -0.3 +0.28j,
          0.08-0.16j, -0.12-0.14j]),
  'feature': array([-0.49701302+0.36852057j, -0.27247295+0.52902364j,
          0.01947328+0.55865332j, ...,  0.16832595+0.52072605j,
          0.42561328+0.35115307j,  0.57760957+0.09470952j])},
 'iq_ro_q2': {'iq_calibration': array([0.5412935 +0.32196122j, 0.03260758-0.00645317j]),
  'coherent_background': array([-0.5 -0.2j , -0.28+0.34j, -0.06+0.5j , ..., -0.3 +0.28j,
          0.08-0.16j, -0.12-0.14j]),
  'feature': array([ 0.57390108+0.31550805j,  0.09636187+0.59491354j,
         -0.45482487+0.41963805j, ...,  0.03022214+0.5985554j ,
         -0.50069715+0.37467876j, -0.64894324-0.13555765j])},
 'iq_ro_q3': {'iq_calibration': array([-0.15177868-0.52652804j,  0.01053425+0.03346687j]),
  'coherent_background': array([-0.5 -0.2j , -0.28+0.34j, -0.06+0.5j , ..., -0.3 +0.2

## усреднённое считывание

In [44]:
modem.filters = modem.calibrated_filters
adc_reducer = data_reduce.data_reduce(modem)
adc_reducer.filters = {'S21_r{}'.format(_qubit_id): 
            data_reduce.mean_reducer(modem, 'iq_ro_q{}'.format(_qubit_id), axis=0) for _qubit_id in qubits.keys()}
adc_reducer.extra_opts['scatter'] = True
adc_reducer.extra_opts['realimag'] = True

In [45]:
adc_reducers = {_qubit_id:data_reduce.data_reduce(modem) for  _qubit_id in qubits.keys()}
for _qubit_id in qubits.keys():
    adc_reducers[_qubit_id].filters = {'S21_r{}'.format(_qubit_id):data_reduce.mean_reducer(
        modem, 'iq_ro_q{}'.format(_qubit_id), axis=0)}
    adc_reducers[_qubit_id].extra_opts['scatter'] = True
    adc_reducers[_qubit_id].extra_opts['realimag'] = True
        
adc_reducer.filters = {'S21_r{}'.format(_qubit_id): 
            data_reduce.mean_reducer(modem, 'iq_ro_q{}'.format(_qubit_id), axis=0) for _qubit_id in qubits.keys()}
adc_reducer.extra_opts['scatter'] = True
adc_reducer.extra_opts['realimag'] = True

## длительность считывания

In [46]:
def set_adc_nop(ro_adc_length):
    adc.stop() # Spectrum
    signal_size = int(np.ceil(4./3.*(ro_adc_length)*adc.get_clock()))
    nop = int( 2**np.ceil(np.log2(signal_size)) )
    #print (nop)
    adc.set_nop(nop)
    adc.set_post_trigger(adc.get_nop()-64)
    
# "Измеритель средних прошедших импульсов"
# Просто усредняет всё по номеру сэмпла.
    
    
#ro_adc_length = 0.15e-6        # Вот такой длины считываем (опухоль Фёдорова!!)
#ro_amplitude =0.16
#ro_dac_length = 0.08e-6
ro_adc_length = 1.0e-6
ro_dac_length = 0.4e-6
#adc.set_nums(1000)
#adc.set_software_nums_multi(1)
adc.set_nums(100)
set_adc_nop(ro_adc_length)
adc.set_nums(5000)
#adc.set_nop(32576)
#adc.set_post_trigger(32576-64)

## Прохождение считывающих испульсов разной амплитуды

In [44]:
#%%mprun -f sweep.sweep
imp.reload(sweep)
imp.reload(plotting)
for _qubit_id in ['1', '2', '3']:
    for nndac_channel_id, channel_voltage in qubits[_qubit_id]['q']['P']['Fmin_coil_currents'].items():
        dac.set_voltage(channel_voltage, channel=nndac_channel_id)
    mean_sample = data_reduce.data_reduce(adc)
    #print (dir(mean_sample))
    #del mean_sample.measure_deferred_result
    mean_sample.filters['Mean Voltage (AC)'] = data_reduce.mean_reducer_noavg(adc, 'Voltage', 0)

    mean_sample.filters['S21+'] = data_reduce.mean_reducer_freq(adc, 'Voltage', 0, iq_ro.carriers['iq_ro_q{}'.format(_qubit_id)].get_if())
    mean_sample.filters['S21-'] = data_reduce.mean_reducer_freq(adc, 'Voltage', 0, -iq_ro.carriers['iq_ro_q{}'.format(_qubit_id)].get_if())
    # # Этот измеритель мы как правило используем когда точек не слишком много и все результаты его жизнедеятельности как правило 
    # # выглядят как ломаные. Чтобы было красиво, давайте лучше сделаем точки (а кривые потом получим фитованые)
    mean_sample.extra_opts['scatter'] = True

    awg.check_cached = True

    mean_sample.filters['Std Voltage (AC)'] = data_reduce.std_reducer_noavg(adc, 'Voltage', 0, 1)
    def set_pulse_amplitude(x):
        awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
        pg.set_seq(trigger_readout_seq+[pg.p('iq_ro_q{}'.format(_qubit_id), ro_dac_length, pg.rect, x)])
        awg_tek.run()
    measurement = sweep.sweep(mean_sample, (np.linspace(0, 1.0, 21), set_pulse_amplitude, 'Амплитуда считывания'), filename='Прохождение считывающих импульсов образец #{}'.format(_qubit_id))
    del mean_sample.filters['Std Voltage (AC)']

HTML(value='')

Started at:  Oct 11 2018 15:05:47
Waiting to join deferred threads:
Exiting sweep main thread #{} via return 12088

Elapsed time: 0 h 0 m 41.71 s


HTML(value='')

Started at:  Oct 11 2018 15:06:39
Waiting to join deferred threads:
Exiting sweep main thread #{} via return 8324

Elapsed time: 0 h 0 m 41.6 s


HTML(value='')

Started at:  Oct 11 2018 15:07:32
Waiting to join deferred threads:
Exiting sweep main thread #{} via return 4656

Elapsed time: 0 h 0 m 41.35 s


In [56]:
zero_response = measurement['Mean Voltage (AC)'][2][0,:]
drive_amplitudes = measurement['Mean Voltage (AC)'][1][0][1:]
signal = measurement['Mean Voltage (AC)'][2][1:,:]
noise = measurement['Std Voltage (AC)'][2][1:,:]
error = noise/np.sqrt(adc.get_nums())
signal_overlap = np.sum(np.conj(signal[0,:])*signal[1:,:], axis=1)/drive_amplitudes[1:]
signal_overlap_estimate = np.real(signal_overlap[0])
signal_overlap_error = 0.5*np.sqrt(np.sum((np.abs(signal[1:,:])*error[0,:])**2, axis=1)+np.sum((np.abs(signal[0,:])*error[1:,:])**2,axis=1))/drive_amplitudes[1:]
#signal_overlap_estimate = (np.sum(np.abs(signal[0,:])**2) - np.sum(error[0,:]*np.abs(signal[0,:]))-np.sum(np.abs(error[0,:])**2))/drive_amplitudes[0]
#plt.figure()
#plt.plot(np.real(signal_overlap))
#plt.plot(np.sum(noise**2, axis=1)/adc.get_nums())
#plt.plot(signal_overlap_error)
compression = 10*np.log10(np.real(signal_overlap)/np.real(signal_overlap_estimate))
db_compression_point = np.argmax(10*np.log10(np.real(signal_overlap)/np.real(signal_overlap_estimate))<-1+10*np.log10(1-signal_overlap_error/signal_overlap_estimate))
#10*np.log10(np.real(signal_overlap)/np.real(signal_overlap_estimate)),-1+10*np.log10(1-signal_overlap_error/signal_overlap_estimate)
#10*np.log10(np.real(signal_overlap)/np.real(signal_overlap_estimate))<-1+10*np.log10(1-signal_overlap_error/signal_overlap_estimate)
if np.any(db_compression_point):
    ro_amplitude = drive_amplitudes[db_compression_point+1]
else:
    ro_amplitude = drive_amplitudes[-1]
print("Readout amplitude:",ro_amplitude)    

Readout amplitude: 1.0


In [40]:
plt.figure('Компрессия считывающего импульса')
plt.plot(drive_amplitudes[1:], compression)
plt.xlabel('Амплитуда считывания')
plt.ylabel('Компрессия, дБ')

Text(13.5972,0.5,'Компрессия, дБ')

In [41]:
ro_parameters = {"adc_len":ro_adc_length,'dac_len': ro_dac_length,'amp': ro_amplitude,'adc nums':adc.nums,
                'trg_length':trg_length}
qjson.dump("setups","readout",ro_parameters)

In [42]:
#ro_amplitude = 0.25
ro_sequence = trigger_readout_seq+[pg.p('iq_ro', ro_dac_length, pg.rect, ro_amplitude)]

## Раби+Рамзи

In [47]:
import qsweepy.qjson as pqjson
import qsweepy.quantum_two_level_dynamics as dyn

In [48]:
imp.reload(dyn)
imp.reload(qjson)
imp.reload(data_reduce)
imp.reload(plotting)
imp.reload(fitting)
imp.reload(sweep)
imp.reload(diff_readout)

<module 'qsweepy.diff_readout' from 'C:\\qtlab_replacement\\qsweepy\\diff_readout.py'>

In [49]:
ex_channels =    ['iq_ex_q{}_F01_min'.format(q_id)                                  for q_id, q in qubits.items()]
ex_amplitudes =  {'iq_ex_q{}_F01_min'.format(q_id):1                                for q_id, q in qubits.items()}
ex_ro_channels = {'iq_ex_q{}_F01_min'.format(q_id):'iq_ro_q{}'.format(q_id)         for q_id, q in qubits.items()}
working_points = {'iq_ex_q{}_F01_min'.format(q_id):q['q']['P']['Fmin_coil_currents'] for q_id, q in qubits.items()}
ro_amplitudes =  {'iq_ex_q1_F01_min':1.0,'iq_ex_q2_F01_min':1.0,'iq_ex_q3_F01_min':1.0}
#coils =          {'iq_ex_q1_F01_min':0,'iq_ex_q2_F01_min':2,'iq_ex_q3_F01_min':4}
coils =          {'iq_ex_q1_F01_min':6,'iq_ex_q2_F01_min':8,'iq_ex_q3_F01_min':10}

rabi_lengths =   {'iq_ex_q{}_F01_min'.format(q_id):np.arange(0., 100e-9, 5e-9)      for q_id, q in qubits.items()}
delaysT2_coherence = {'iq_ex_q{}_F01_min'.format(q_id):np.arange(0., 5e-6, 25e-9)   for q_id, q in qubits.items()}
#delaysT2_coherence = {'iq_ex_q{}_F01_min'.format(q_id):np.arange(0, 200e-9, 2e-9)   for q_id, q in qubits.items()}
fitters = {}
adc_reducers_chan = {}
for q_id, q in qubits.items():
    ro_meas_name = 'S21_r{}'.format(q_id)
    adc_reducers_chan['iq_ex_q{}_F01_min'.format(q_id)] = adc_reducers[q_id]
    fitters['iq_ex_q{}_F01_min'.format(q_id)] = lambda x,y,ro_meas_name=ro_meas_name: fitting.fit1d(x,y,ro_meas_name)

#rabi_lengths = np.arange(0., 2000e-9, 5e-9)
delaysT2 = np.arange(0, 400e-9, 4e-9)
target_freq_offset = -30e6
#delaysT2_coherence = np.arange(0, 5000e-9, 10e-9)
target_freq_offset_coherence = -3e6
delaysT1 = np.arange(0, 10e-6,0.1e-6)

In [50]:
diff_adc_reducer = diff_readout.diff_readout(adc_reducer)

In [51]:
twos = {ex_channel:dyn.quantum_two_level_dynamics(pg,
                                           adc_reducers_chan[ex_channel],
                                           ex_channel=ex_channel,
                                           ro_channel=ex_ro_channels[ex_channel],
                                           ro_sequence=(trigger_readout_seq+\
                            [pg.p(ex_ro_channels[ex_channel], ro_dac_length, pg.rect, ro_amplitudes[ex_channel])]),
                                           ex_amplitude=ex_amplitudes[ex_channel],
                                           fitter=fitters[ex_channel], 
                                           shuffle=False,
                                           qubit_id=qubit_id) for ex_channel in ex_channels}
#twos = {'iq_ex_q3_F01_min':dyn.quantum_two_level_dynamics(pg,
#                                           adc_reducer,
#                                           ex_channel='iq_ex_q3_F01_min',
#                                           ro_channel=ex_ro_channels['iq_ex_q3_F01_min'],
#                                           ro_sequence=(trigger_readout_seq+\
#                            [pg.p(ex_ro_channels['iq_ex_q3_F01_min'], ro_dac_length, pg.rect, ro_amplitudes['iq_ex_q3_F01_min'])]),
#                                           ex_amplitude=ex_amplitudes['iq_ex_q3_F01_min'],
#                                           fitter=fitters['iq_ex_q3_F01_min'], 
#                                           shuffle=True,
#                                           qubit_id=qubit_id)}

c:\data/calibrations/setups/ readout
Failed loading readout calibration: max() arg is an empty sequence
Failed loading rabi frequency calibration: 'quantum_two_level_dynamics' object has no attribute 'ex_amplitude'
c:\data/calibrations/setups/ readout
Failed loading readout calibration: max() arg is an empty sequence
Failed loading rabi frequency calibration: 'quantum_two_level_dynamics' object has no attribute 'ex_amplitude'
c:\data/calibrations/setups/ readout
Failed loading readout calibration: max() arg is an empty sequence
Failed loading rabi frequency calibration: 'quantum_two_level_dynamics' object has no attribute 'ex_amplitude'


In [53]:
#%%mprun -f two_diffs[0].Rabi_rect -f two_diffs[0].Ramsey -f two_diffs[0].decay -f two_diffs[0].spin_echo -f sweep.sweep -f diff_adc_reducer.measure
#dac.set_voltage(1.772, channel=0)

two_diffs = twos
imp.reload(plotting)
iters = 1
for _iter in range(iters):
    for ex_channel, two_diff in two_diffs.items():
        dac = nndac('TCPIP0::10.20.61.12::1000::SOCKET')
        if _iter==0 and (ex_channel == 'iq_ex_q1_F01_min'):# or ex_channel == 'iq_ex_q2_F01_min'):
            continue
        for nndac_channel_id, channel_voltage in working_points[ex_channel].items():
            dac.set_voltage(channel_voltage, channel=nndac_channel_id)
        coil = coils[ex_channel]
        voltages = working_points[ex_channel][coil]+np.linspace(-0.025, 0.025, 11)
        #plt.close('all')

        two_diff.Rabi_rect(rabi_lengths[ex_channel])
#         two_diff.Ramsey(delaysT2, target_freq_offset)
        two_diff.Ramsey(delaysT2_coherence[ex_channel],target_freq_offset_coherence)
#         two_diff.decay(delaysT1)
        #two_diff.spin_echo(delaysT2_coherence[ex_channel],target_freq_offset_coherence)
        #plt.close('all')
        
        #two_diff.Ramsey(delaysT2_coherence[ex_channel],target_freq_offset_coherence, 
        #            (voltages, lambda x: dac.set_voltage(x, channel=coil), 'Coil #{} voltage'.format(coil)))
        #two_diff.decay(delaysT1, 
        #            (voltages, lambda x: dac.set_voltage(x, channel=coil), 'Coil #{} voltage'.format(coil)))
        #two_diff.spin_echo(delaysT2_coherence[ex_channel],target_freq_offset_coherence, 
        #            (voltages, lambda x: dac.set_voltage(x, channel=coil), 'Coil #{} voltage'.format(coil)))

HTML(value='')

Started at:  Oct 12 2018 08:53:23
Waiting to join deferred threads:
Exiting sweep main thread #{} via return 8552

Elapsed time: 0 h 0 m 49.07 s


HTML(value='')

Started at:  Oct 12 2018 08:54:15


Exception in thread Thread-34:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "C:\qtlab_replacement\qsweepy\sweep.py", line 281, in main_sweep_loop
    set_single_measurement_result(mpoint, indeces)
  File "C:\qtlab_replacement\qsweepy\sweep.py", line 225, in set_single_measurement_result
    ascii_file.write(data_to_str(fmt(mpoint_m_flat))+'\n')
ValueError: I/O operation on closed file.



Error: Cannot move a directory 'c:\data/2018-10-12/08-54-15-Ramsey (target offset -3.00 MHz), excitation channel iq_ex_q2_F01_min/' into itself 'c:\data/2018-10-12/08-54-15-Ramsey (target offset -3.00 MHz), excitation channel iq_ex_q2_F01_min/unfinished/'.

In [198]:
plt.close('all')

In [20]:
dac

In [231]:
import qsweepy.save_pkl
import pandas as pd
measurement_folders = {'qubit1_1.93':
               {'paths': [r'D:\data\2018-09-22\03-24-38-Ramsey (target offset -3.00 MHz), excitation channel iq_ex_q1_F01_min',
                r'D:\data\2018-09-22\04-55-31-Ramsey (target offset -3.00 MHz), excitation channel iq_ex_q1_F01_min',
                r'D:\data\2018-09-22\09-06-10-Ramsey (target offset -3.00 MHz), excitation channel iq_ex_q1_F01_min',
                r'D:\data\2018-09-22\09-37-06-Ramsey (target offset -3.00 MHz), excitation channel iq_ex_q1_F01_min'],
                'name': 'Ramsey (target offset -3.00 MHz), excitation channel iq_ex_q1_F01_min', 'q': '1'},
                       'qubit1_1.80':
               {'paths': [r'D:\data\2018-09-22\10-07-49-Ramsey (target offset -3.00 MHz), excitation channel iq_ex_q1_F01_min',
                          r'D:\data\2018-09-22\10-38-28-Ramsey (target offset -3.00 MHz), excitation channel iq_ex_q1_F01_min',
                          r'D:\data\2018-09-22\11-09-04-Ramsey (target offset -3.00 MHz), excitation channel iq_ex_q1_F01_min'],
                'name': 'Ramsey (target offset -3.00 MHz), excitation channel iq_ex_q1_F01_min', 'q': '1'},
                       'qubit1_1.79':
               {'paths': [r'D:\data\2018-09-22\11-57-52-Ramsey (target offset -3.00 MHz), excitation channel iq_ex_q1_F01_min'],
                'name': 'Ramsey (target offset -3.00 MHz), excitation channel iq_ex_q1_F01_min', 'q': '1'}}
#measurement = qsweepy.save_pkl.load_pkl('Ramsey (target offset -3.00 MHz), excitation channel iq_ex_q1_F01_min', r'D:\data\2018-09-22\03-24-38-Ramsey (target offset -3.00 MHz), excitation channel iq_ex_q1_F01_min')[1]
#measurement = qsweepy.save_pkl.load_pkl('Ramsey (target offset -3.00 MHz), excitation channel iq_ex_q1_F01_min', r'D:\data\2018-09-22\04-55-31-Ramsey (target offset -3.00 MHz), excitation channel iq_ex_q1_F01_min')[1]


#measurement = qsweepy.save_pkl.load_pkl('Ramsey (target offset -3.00 MHz), excitation channel iq_ex_q2_F01_min', r'D:\data\2018-09-22\03-54-59-Ramsey (target offset -3.00 MHz), excitation channel iq_ex_q2_F01_min')[1]
#measurement = qsweepy.save_pkl.load_pkl('Ramsey (target offset -3.00 MHz), excitation channel iq_ex_q2_F01_min', r'D:\data\2018-09-22\05-26-00-Ramsey (target offset -3.00 MHz), excitation channel iq_ex_q2_F01_min')[1]

#measurement = qsweepy.save_pkl.load_pkl('Ramsey (target offset -3.00 MHz), excitation channel iq_ex_q3_F01_min', r'D:\data\2018-09-22\04-25-11-Ramsey (target offset -3.00 MHz), excitation channel iq_ex_q3_F01_min')[1]
#measurement = qsweepy.save_pkl.load_pkl('Ramsey (target offset -3.00 MHz), excitation channel iq_ex_q3_F01_min', r'D:\data\2018-09-22\05-56-20-Ramsey (target offset -3.00 MHz), excitation channel iq_ex_q3_F01_min')[1]

In [232]:
for measurement_type_name, measurement_type in measurement_folders.items(): 
    for measurement_folder in measurement_type['paths']:
        measurement = qsweepy.save_pkl.load_pkl(measurement_type['name'], measurement_folder)[1]
        q = measurement_type['q']
        summary = pd.DataFrame([dict(fitting.exp_sin_fit(measurement['S21_r'+q][1][0], 
                         [np.real(measurement['S21_r'+q][2][:,dc_id]),
                          np.imag(measurement['S21_r'+q][2][:,dc_id])])[-1], **{'dc':dc}) 
                             for dc_id, dc in enumerate(measurement['S21_r'+q][1][1])])
        plt.figure(measurement_type_name+' Ramsey frequency')
        plt.plot(np.asarray(summary['dc']), np.asarray(summary['freq'])/1e6)
        plt.grid()
        plt.xlabel('Control voltage, V')
        plt.ylabel('Ramsey frequency, MHz')
        plt.title(measurement_type_name+' Ramsey frequency')
        plt.tight_layout()
        plt.figure(measurement_type_name+' Ramsey decay')
        plt.plot(np.asarray(summary['dc']), np.asarray(summary['decay'])*1e6)
        plt.grid()
        plt.xlabel('Control voltage, V')
        plt.ylabel('Ramsey decay, $\\mu$s')
        plt.title(measurement_type_name+' Ramsey decay')
        plt.tight_layout()

D:\data\2018-09-22\03-24-38-Ramsey (target offset -3.00 MHz), excitation channel iq_ex_q1_F01_min/Ramsey (target offset -3.00 MHz), excitation channel iq_ex_q1_F01_min.pkl
D:\data\2018-09-22\04-55-31-Ramsey (target offset -3.00 MHz), excitation channel iq_ex_q1_F01_min/Ramsey (target offset -3.00 MHz), excitation channel iq_ex_q1_F01_min.pkl
D:\data\2018-09-22\09-06-10-Ramsey (target offset -3.00 MHz), excitation channel iq_ex_q1_F01_min/Ramsey (target offset -3.00 MHz), excitation channel iq_ex_q1_F01_min.pkl
D:\data\2018-09-22\09-37-06-Ramsey (target offset -3.00 MHz), excitation channel iq_ex_q1_F01_min/Ramsey (target offset -3.00 MHz), excitation channel iq_ex_q1_F01_min.pkl
D:\data\2018-09-22\10-07-49-Ramsey (target offset -3.00 MHz), excitation channel iq_ex_q1_F01_min/Ramsey (target offset -3.00 MHz), excitation channel iq_ex_q1_F01_min.pkl
D:\data\2018-09-22\10-38-28-Ramsey (target offset -3.00 MHz), excitation channel iq_ex_q1_F01_min/Ramsey (target offset -3.00 MHz), excitati